DISPATCHING RULE - EDD

In [3]:
import pandas as pd
import numpy as np
import heapq
import os

data_path = '../data'

data = pd.read_excel(os.path.join(data_path, 'Data.xlsx'), sheet_name=0, index_col=0)
data = data.reset_index()
data['Start time 1'] = 0
data['Finish time 1'] = 0
data['Máy trải'] = 0
def schedule_jobs(df, machines):
    # Sắp xếp công việc theo thời gian tới hạn (due date) tăng dần
    sorted_jobs = data.sort_values(by='Due date (h)').values

    # Khởi tạo hàng đợi ưu tiên (min heap) để theo dõi thời gian hoàn thành trên từng máy
    machines_heap = [(0, machine_id) for machine_id in range(machines)]

    # Lập lịch công việc
    for job in sorted_jobs:
        processing_time, machine_id = heapq.heappop(machines_heap)
        start_time=0
        start_time = max(processing_time,start_time)  # Thời điểm bắt đầu là max(thời gian hoàn thành trước đó, thời gian bắt đầu công việc)
        finish_time = start_time + job[5]  # Thời gian hoàn thành công việc là thời gian bắt đầu + thời gian gia công
        df.at[int(job[0]) - 1, 'Start time 1'] = start_time
        df.at[int(job[0]) - 1, 'Finish time 1'] = finish_time
        df.at[int(job[0]) - 1, 'Máy trải'] = machine_id + 1
        heapq.heappush(machines_heap, (finish_time, machine_id))  # Cập nhật hàng đợi với thời gian hoàn thành mới

        #print(f"Công việc {int(job[0])} được lập lịch cho máy {machine_id + 1} từ {start_time} đến {finish_time}")
# Số lượng máy
số_máy_trải= 8

schedule_jobs(data, số_máy_trải)
data_stage2= data.sort_values(by='Máy trải')
display(data_stage2)
#Gán việc cho máy cắt
cutting_machine = []
for ind in data_stage2.index:
    cutting_machine.append((data_stage2['Máy trải'][ind] + 1) // 2)

data_stage2['Máy cắt'] = cutting_machine
data_stage2 = data_stage2.sort_values(by=['Finish time 1', 'Máy cắt'])
display(data_stage2)
#Tính thời gian cho trạm cắt
num_cutting_machine = 4
cur_cutting_time: list[float] = [data_stage2[data_stage2['Máy cắt'] == idx + 1].iloc[0, 8] for idx in range(num_cutting_machine)]

data_stage2['Start time 2'] = 0.0
data_stage2['Finish time 2'] = 0.0

for ind in data_stage2.index:
    cutting_idx = data_stage2.at[ind, 'Máy cắt'] - 1
    data_stage2.at[ind, 'Start time 2'] = max(cur_cutting_time[cutting_idx], data_stage2.at[ind, 'Finish time 1'])
    data_stage2.at[ind, 'Finish time 2'] = data_stage2['Start time 2'][ind] + data_stage2['P cắt'][ind]

    # Update current cutting time
    cur_cutting_time[cutting_idx] = data_stage2.at[ind, 'Finish time 2']
#Tính độ trễ
data_stage2['Độ trễ (phút)']= np.maximum(0, data_stage2['Finish time 2'] - (data_stage2['Due date (h)'] * 60))
print('BẢNG KẾT QUẢ ĐIỀU ĐỘ TRẠM MÁY CẮT')
display(data_stage2)

#Tính hàm mục tiêu
column_count = 'Độ trễ (phút)'
num_tardiness = (data_stage2[column_count] > 0).sum()
print('Số công việc bị trễ theo luật EDD = ', num_tardiness)
sum_tardiness = data_stage2['Độ trễ (phút)'].sum() 
print('Hàm mục tiêu theo luật EDD = ',sum_tardiness,'phút')

# Thứ tự điều độ tại từng trạm
data_stage2
job_order_start_time_1 = data_stage2.sort_values(by='Start time 1')
print("\nKết quả thứ tự điều độ trên trạm trải:")
display(job_order_start_time_1[['Jobs', 'Start time 1']])

job_order_start_time_2 = data_stage2.sort_values(by='Start time 2')
print("\nKết quả thứ tự điều độ trên trạm cắt:")
display(job_order_start_time_2[['Jobs', 'Start time 2']])

/var/folders/4q/62nlmhj92rsbpt7h4b7nlmdr0000gn/T/ipykernel_2921/3023565608.py:27: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '93.07' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[int(job[0]) - 1, 'Finish time 1'] = finish_time
/var/folders/4q/62nlmhj92rsbpt7h4b7nlmdr0000gn/T/ipykernel_2921/3023565608.py:26: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '15.07' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[int(job[0]) - 1, 'Start time 1'] = start_time


,Jobs,Stockout Num,Total rolls,Total yards,Due date (h),P trải,P cắt,Start time 1,Finish time 1,Máy trải
9,10,20230720024146,10,698,2,93.07,48.0,0.00,93.07,1
16,17,20230720024153,7,225,9,30.00,33.6,93.07,123.07,1
15,16,20230720024152,8,251,2,33.47,38.4,0.00,33.47,2
5,6,20230720024142,5,354,7,47.20,24.0,75.47,122.67,2
11,12,20230720024148,6,315,4,42.00,28.8,33.47,75.47,2
2,3,20230720024139,6,486,4,64.80,28.8,36.67,101.47,3
13,14,20230720024150,5,275,2,36.67,24.0,0.00,36.67,3
1,2,20230720024138,7,354,2,47.20,33.6,15.07,62.27,4
7,8,20230720024144,10,856,7,114.14,48.0,62.27,176.41,4
8,9,20230720024145,5,113,2,15.07,24.0,0.00,15.07,4


,Jobs,Stockout Num,Total rolls,Total yards,Due date (h),P trải,P cắt,Start time 1,Finish time 1,Máy trải,Máy cắt
8,9,20230720024145,5,113,2,15.07,24.0,0.00,15.07,4,2
15,16,20230720024152,8,251,2,33.47,38.4,0.00,33.47,2,1
13,14,20230720024150,5,275,2,36.67,24.0,0.00,36.67,3,2
3,4,20230720024140,5,337,2,44.94,24.0,0.00,44.94,8,4
4,5,20230720024141,7,406,2,54.14,33.6,0.00,54.14,7,4
1,2,20230720024138,7,354,2,47.20,33.6,15.07,62.27,4,2
19,20,20230720024156,9,543,2,72.40,43.2,0.00,72.40,6,3
11,12,20230720024148,6,315,4,42.00,28.8,33.47,75.47,2,1
6,7,20230720024143,8,587,2,78.27,38.4,0.00,78.27,5,3
17,18,20230720024154,9,332,4,44.27,43.2,44.94,89.21,8,4


BẢNG KẾT QUẢ ĐIỀU ĐỘ TRẠM MÁY CẮT


,Jobs,Stockout Num,Total rolls,Total yards,Due date (h),P trải,P cắt,Start time 1,Finish time 1,Máy trải,Máy cắt,Start time 2,Finish time 2,Độ trễ (phút)
8,9,20230720024145,5,113,2,15.07,24.0,0.00,15.07,4,2,15.07,39.07,0.00
15,16,20230720024152,8,251,2,33.47,38.4,0.00,33.47,2,1,33.47,71.87,0.00
13,14,20230720024150,5,275,2,36.67,24.0,0.00,36.67,3,2,39.07,63.07,0.00
3,4,20230720024140,5,337,2,44.94,24.0,0.00,44.94,8,4,44.94,68.94,0.00
4,5,20230720024141,7,406,2,54.14,33.6,0.00,54.14,7,4,68.94,102.54,0.00
1,2,20230720024138,7,354,2,47.20,33.6,15.07,62.27,4,2,63.07,96.67,0.00
19,20,20230720024156,9,543,2,72.40,43.2,0.00,72.40,6,3,72.40,115.60,0.00
11,12,20230720024148,6,315,4,42.00,28.8,33.47,75.47,2,1,75.47,104.27,0.00
6,7,20230720024143,8,587,2,78.27,38.4,0.00,78.27,5,3,115.60,154.00,34.00
17,18,20230720024154,9,332,4,44.27,43.2,44.94,89.21,8,4,102.54,145.74,0.00


Số công việc bị trễ theo luật EDD =  2
Hàm mục tiêu theo luật EDD =  66.26999999999998 phút

Kết quả thứ tự điều độ trên trạm trải:


,Jobs,Start time 1
8,9,0.00
15,16,0.00
13,14,0.00
3,4,0.00
4,5,0.00
19,20,0.00
6,7,0.00
9,10,0.00
1,2,15.07
11,12,33.47



Kết quả thứ tự điều độ trên trạm cắt:


,Jobs,Start time 2
8,9,15.07
15,16,33.47
13,14,39.07
3,4,44.94
1,2,63.07
4,5,68.94
19,20,72.40
11,12,75.47
2,3,101.47
17,18,102.54


DISPATCHING RULE - FCFS

In [41]:
import pandas as pd
import numpy as np
import heapq
import os
data = pd.read_excel(os.path.join(data_path, 'Data.xlsx'), sheet_name=0, index_col=0)
data = data.reset_index()
data['Start time 1'] = 0
data['Finish time 1'] = 0
data['Máy trải'] = 0
def schedule_jobs(df, machines):
    # Sắp xếp công việc theo thời gian tới hạn (due date) tăng dần
    sorted_jobs = data.sort_values(by='Jobs').values

    # Khởi tạo hàng đợi ưu tiên (min heap) để theo dõi thời gian hoàn thành trên từng máy
    machines_heap = [(0, machine_id) for machine_id in range(machines)]

    # Lập lịch công việc
    for job in sorted_jobs:
        processing_time, machine_id = heapq.heappop(machines_heap)
        start_time=0
        start_time = max(processing_time,start_time)  # Thời điểm bắt đầu là max(thời gian hoàn thành trước đó, thời gian bắt đầu công việc)
        finish_time = start_time + job[5]  # Thời gian hoàn thành công việc là thời gian bắt đầu + thời gian gia công
        df.at[int(job[0]) - 1, 'Start time 1'] = start_time
        df.at[int(job[0]) - 1, 'Finish time 1'] = finish_time
        df.at[int(job[0]) - 1, 'Máy trải'] = machine_id + 1
        heapq.heappush(machines_heap, (finish_time, machine_id))  # Cập nhật hàng đợi với thời gian hoàn thành mới

        #print(f"Công việc {int(job[0])} được lập lịch cho máy {machine_id + 1} từ {start_time} đến {finish_time}")
# Số lượng máy
số_máy_trải= 8

schedule_jobs(data, số_máy_trải)
data_stage2= data.sort_values(by='Máy trải')
display(data_stage2)
#Gán việc cho máy cắt
cutting_machine = []
for ind in data_stage2.index:
    cutting_machine.append((data_stage2['Máy trải'][ind] + 1) // 2)

data_stage2['Máy cắt'] = cutting_machine
data_stage2 = data_stage2.sort_values(by=['Finish time 1', 'Máy cắt'])
display(data_stage2)
#Tính thời gian cho trạm cắt
num_cutting_machine = 4
cur_cutting_time: list[float] = [data_stage2[data_stage2['Máy cắt'] == idx + 1].iloc[0, 8] for idx in range(num_cutting_machine)]

data_stage2['Start time 2'] = 0.0
data_stage2['Finish time 2'] = 0.0

for ind in data_stage2.index:
    cutting_idx = data_stage2.at[ind, 'Máy cắt'] - 1
    data_stage2.at[ind, 'Start time 2'] = max(cur_cutting_time[cutting_idx], data_stage2.at[ind, 'Finish time 1'])
    data_stage2.at[ind, 'Finish time 2'] = data_stage2['Start time 2'][ind] + data_stage2['P cắt'][ind]

    # Update current cutting time
    cur_cutting_time[cutting_idx] = data_stage2.at[ind, 'Finish time 2']
#Tính độ trễ
data_stage2['Độ trễ (phút)']= np.maximum(0, data_stage2['Finish time 2'] - (data_stage2['Due date (h)'] * 60))
print('BẢNG KẾT QUẢ ĐIỀU ĐỘ TRẠM MÁY CẮT')
display(data_stage2)

#Tính hàm mục tiêu
column_count = 'Độ trễ (phút)'
num_tardiness = (data_stage2[column_count] > 0).sum()
print('Số công việc bị trễ theo luật FCFS = ', num_tardiness)
sum_tardiness = data_stage2['Độ trễ (phút)'].sum() 
print('Hàm mục tiêu theo luật FCFS = ',sum_tardiness,'phút')

# Thứ tự điều độ tại từng trạm
data_stage2
job_order_start_time_1 = data_stage2.sort_values(by='Start time 1')
print("\nKết quả thứ tự điều độ trên trạm trải:")
display(job_order_start_time_1[['Jobs', 'Start time 1']])

job_order_start_time_2 = data_stage2.sort_values(by='Start time 2')
print("\nKết quả thứ tự điều độ trên trạm cắt:")
display(job_order_start_time_2[['Jobs', 'Start time 2']])

,Jobs,Stockout Num,Total rolls,Total yards,Due date (h),P trải,P cắt,Start time 1,Finish time 1,Máy trải
0,1,20230720024137,10,646,4,86.14,48.0,0.00,86.14,1
17,18,20230720024154,9,332,4,44.27,43.2,86.14,130.41,1
9,10,20230720024146,10,698,2,93.07,48.0,47.20,140.27,2
1,2,20230720024138,7,354,2,47.20,33.6,0.00,47.20,2
13,14,20230720024150,5,275,2,36.67,24.0,64.80,101.47,3
2,3,20230720024139,6,486,4,64.80,28.8,0.00,64.80,3
3,4,20230720024140,5,337,2,44.94,24.0,0.00,44.94,4
12,13,20230720024149,6,196,9,26.14,28.8,60.01,86.15,4
8,9,20230720024145,5,113,2,15.07,24.0,44.94,60.01,4
18,19,20230720024155,7,720,9,96.00,33.6,86.15,182.15,4


,Jobs,Stockout Num,Total rolls,Total yards,Due date (h),P trải,P cắt,Start time 1,Finish time 1,Máy trải,Máy cắt
3,4,20230720024140,5,337,2,44.94,24.0,0.00,44.94,4,2
1,2,20230720024138,7,354,2,47.20,33.6,0.00,47.20,2,1
5,6,20230720024142,5,354,7,47.20,24.0,0.00,47.20,6,3
4,5,20230720024141,7,406,2,54.14,33.6,0.00,54.14,5,3
8,9,20230720024145,5,113,2,15.07,24.0,44.94,60.01,4,2
2,3,20230720024139,6,486,4,64.80,28.8,0.00,64.80,3,2
10,11,20230720024147,5,140,7,18.67,24.0,47.20,65.87,6,3
6,7,20230720024143,8,587,2,78.27,38.4,0.00,78.27,7,4
14,15,20230720024151,5,144,9,19.20,24.0,65.87,85.07,6,3
0,1,20230720024137,10,646,4,86.14,48.0,0.00,86.14,1,1


BẢNG KẾT QUẢ ĐIỀU ĐỘ TRẠM MÁY CẮT


,Jobs,Stockout Num,Total rolls,Total yards,Due date (h),P trải,P cắt,Start time 1,Finish time 1,Máy trải,Máy cắt,Start time 2,Finish time 2,Độ trễ (phút)
3,4,20230720024140,5,337,2,44.94,24.0,0.00,44.94,4,2,44.94,68.94,0.00
1,2,20230720024138,7,354,2,47.20,33.6,0.00,47.20,2,1,47.20,80.80,0.00
5,6,20230720024142,5,354,7,47.20,24.0,0.00,47.20,6,3,47.20,71.20,0.00
4,5,20230720024141,7,406,2,54.14,33.6,0.00,54.14,5,3,71.20,104.80,0.00
8,9,20230720024145,5,113,2,15.07,24.0,44.94,60.01,4,2,68.94,92.94,0.00
2,3,20230720024139,6,486,4,64.80,28.8,0.00,64.80,3,2,92.94,121.74,0.00
10,11,20230720024147,5,140,7,18.67,24.0,47.20,65.87,6,3,104.80,128.80,0.00
6,7,20230720024143,8,587,2,78.27,38.4,0.00,78.27,7,4,78.27,116.67,0.00
14,15,20230720024151,5,144,9,19.20,24.0,65.87,85.07,6,3,128.80,152.80,0.00
0,1,20230720024137,10,646,4,86.14,48.0,0.00,86.14,1,1,86.14,134.14,0.00


Số công việc bị trễ theo luật FCFS =  4
Hàm mục tiêu theo luật FCFS =  333.35 phút

Kết quả thứ tự điều độ trên trạm trải:


,Jobs,Start time 1
3,4,0.00
7,8,0.00
6,7,0.00
2,3,0.00
0,1,0.00
4,5,0.00
5,6,0.00
1,2,0.00
8,9,44.94
10,11,47.20



Kết quả thứ tự điều độ trên trạm cắt:


,Jobs,Start time 2
3,4,44.94
1,2,47.20
5,6,47.20
8,9,68.94
4,5,71.20
6,7,78.27
0,1,86.14
2,3,92.94
10,11,104.80
15,16,116.67


DISPATCHING RULE - SPT

In [42]:
import pandas as pd
import numpy as np
import heapq
import os
data = pd.read_excel(os.path.join(data_path, 'Data.xlsx'), sheet_name=0, index_col=0)
data = data.reset_index()
data['Start time 1'] = 0
data['Finish time 1'] = 0
data['Máy trải'] = 0
def schedule_jobs(df, machines):
    # Sắp xếp công việc theo thời gian tới hạn (due date) tăng dần
    sorted_jobs = data.sort_values(by='P trải').values

    # Khởi tạo hàng đợi ưu tiên (min heap) để theo dõi thời gian hoàn thành trên từng máy
    machines_heap = [(0, machine_id) for machine_id in range(machines)]

    # Lập lịch công việc
    for job in sorted_jobs:
        processing_time, machine_id = heapq.heappop(machines_heap)
        start_time=0
        start_time = max(processing_time,start_time)  # Thời điểm bắt đầu là max(thời gian hoàn thành trước đó, thời gian bắt đầu công việc)
        finish_time = start_time + job[5]  # Thời gian hoàn thành công việc là thời gian bắt đầu + thời gian gia công
        df.at[int(job[0]) - 1, 'Start time 1'] = start_time
        df.at[int(job[0]) - 1, 'Finish time 1'] = finish_time
        df.at[int(job[0]) - 1, 'Máy trải'] = machine_id + 1
        heapq.heappush(machines_heap, (finish_time, machine_id))  # Cập nhật hàng đợi với thời gian hoàn thành mới

        #print(f"Công việc {int(job[0])} được lập lịch cho máy {machine_id + 1} từ {start_time} đến {finish_time}")
# Số lượng máy
số_máy_trải= 8

schedule_jobs(data, số_máy_trải)
data_stage2= data.sort_values(by='Máy trải')
display(data_stage2)
#Gán việc cho máy cắt
cutting_machine = []
for ind in data_stage2.index:
    cutting_machine.append((data_stage2['Máy trải'][ind] + 1) // 2)

data_stage2['Máy cắt'] = cutting_machine
data_stage2 = data_stage2.sort_values(by=['Finish time 1', 'Máy cắt'])
display(data_stage2)
#Tính thời gian cho trạm cắt
num_cutting_machine = 4
cur_cutting_time: list[float] = [data_stage2[data_stage2['Máy cắt'] == idx + 1].iloc[0, 8] for idx in range(num_cutting_machine)]

data_stage2['Start time 2'] = 0.0
data_stage2['Finish time 2'] = 0.0

for ind in data_stage2.index:
    cutting_idx = data_stage2.at[ind, 'Máy cắt'] - 1
    data_stage2.at[ind, 'Start time 2'] = max(cur_cutting_time[cutting_idx], data_stage2.at[ind, 'Finish time 1'])
    data_stage2.at[ind, 'Finish time 2'] = data_stage2['Start time 2'][ind] + data_stage2['P cắt'][ind]

    # Update current cutting time
    cur_cutting_time[cutting_idx] = data_stage2.at[ind, 'Finish time 2']
#Tính độ trễ
data_stage2['Độ trễ (phút)']= np.maximum(0, data_stage2['Finish time 2'] - (data_stage2['Due date (h)'] * 60))
print('BẢNG KẾT QUẢ ĐIỀU ĐỘ TRẠM MÁY CẮT')
display(data_stage2)

#Tính hàm mục tiêu
column_count = 'Độ trễ (phút)'
num_tardiness = (data_stage2[column_count] > 0).sum()
print('Số công việc bị trễ theo luật SPT = ', num_tardiness)
sum_tardiness = data_stage2['Độ trễ (phút)'].sum() 
print('Hàm mục tiêu theo luật SPT = ',sum_tardiness,'phút')

# Thứ tự điều độ tại từng trạm
data_stage2
job_order_start_time_1 = data_stage2.sort_values(by='Start time 1')
print("\nKết quả thứ tự điều độ trên trạm trải:")
display(job_order_start_time_1[['Jobs', 'Start time 1']])

job_order_start_time_2 = data_stage2.sort_values(by='Start time 2')
print("\nKết quả thứ tự điều độ trên trạm cắt:")
display(job_order_start_time_2[['Jobs', 'Start time 2']])

,Jobs,Stockout Num,Total rolls,Total yards,Due date (h),P trải,P cắt,Start time 1,Finish time 1,Máy trải
0,1,20230720024137,10,646,4,86.14,48.0,59.34,145.48,1
17,18,20230720024154,9,332,4,44.27,43.2,15.07,59.34,1
8,9,20230720024145,5,113,2,15.07,24.0,0.00,15.07,1
10,11,20230720024147,5,140,7,18.67,24.0,0.00,18.67,2
9,10,20230720024146,10,698,2,93.07,48.0,63.61,156.68,2
3,4,20230720024140,5,337,2,44.94,24.0,18.67,63.61,2
18,19,20230720024155,7,720,9,96.00,33.6,66.40,162.40,3
14,15,20230720024151,5,144,9,19.20,24.0,0.00,19.20,3
1,2,20230720024138,7,354,2,47.20,33.6,19.20,66.40,3
5,6,20230720024142,5,354,7,47.20,24.0,26.14,73.34,4


,Jobs,Stockout Num,Total rolls,Total yards,Due date (h),P trải,P cắt,Start time 1,Finish time 1,Máy trải,Máy cắt
8,9,20230720024145,5,113,2,15.07,24.0,0.00,15.07,1,1
10,11,20230720024147,5,140,7,18.67,24.0,0.00,18.67,2,1
14,15,20230720024151,5,144,9,19.20,24.0,0.00,19.20,3,2
12,13,20230720024149,6,196,9,26.14,28.8,0.00,26.14,4,2
16,17,20230720024153,7,225,9,30.00,33.6,0.00,30.00,5,3
15,16,20230720024152,8,251,2,33.47,38.4,0.00,33.47,6,3
13,14,20230720024150,5,275,2,36.67,24.0,0.00,36.67,7,4
11,12,20230720024148,6,315,4,42.00,28.8,0.00,42.00,8,4
17,18,20230720024154,9,332,4,44.27,43.2,15.07,59.34,1,1
3,4,20230720024140,5,337,2,44.94,24.0,18.67,63.61,2,1


BẢNG KẾT QUẢ ĐIỀU ĐỘ TRẠM MÁY CẮT


,Jobs,Stockout Num,Total rolls,Total yards,Due date (h),P trải,P cắt,Start time 1,Finish time 1,Máy trải,Máy cắt,Start time 2,Finish time 2,Độ trễ (phút)
8,9,20230720024145,5,113,2,15.07,24.0,0.00,15.07,1,1,15.07,39.07,0.00
10,11,20230720024147,5,140,7,18.67,24.0,0.00,18.67,2,1,39.07,63.07,0.00
14,15,20230720024151,5,144,9,19.20,24.0,0.00,19.20,3,2,19.20,43.20,0.00
12,13,20230720024149,6,196,9,26.14,28.8,0.00,26.14,4,2,43.20,72.00,0.00
16,17,20230720024153,7,225,9,30.00,33.6,0.00,30.00,5,3,30.00,63.60,0.00
15,16,20230720024152,8,251,2,33.47,38.4,0.00,33.47,6,3,63.60,102.00,0.00
13,14,20230720024150,5,275,2,36.67,24.0,0.00,36.67,7,4,36.67,60.67,0.00
11,12,20230720024148,6,315,4,42.00,28.8,0.00,42.00,8,4,60.67,89.47,0.00
17,18,20230720024154,9,332,4,44.27,43.2,15.07,59.34,1,1,63.07,106.27,0.00
3,4,20230720024140,5,337,2,44.94,24.0,18.67,63.61,2,1,106.27,130.27,10.27


Số công việc bị trễ theo luật SPT =  5
Hàm mục tiêu theo luật SPT =  250.29000000000005 phút

Kết quả thứ tự điều độ trên trạm trải:


,Jobs,Start time 1
8,9,0.00
10,11,0.00
14,15,0.00
12,13,0.00
16,17,0.00
15,16,0.00
13,14,0.00
11,12,0.00
17,18,15.07
3,4,18.67



Kết quả thứ tự điều độ trên trạm cắt:


,Jobs,Start time 2
8,9,15.07
14,15,19.20
16,17,30.00
13,14,36.67
10,11,39.07
12,13,43.20
11,12,60.67
17,18,63.07
15,16,63.60
1,2,72.00


DISPATCHING RULE - LPT

In [43]:
import pandas as pd
import numpy as np
import heapq
import os
data = pd.read_excel(os.path.join(data_path, 'Data.xlsx'), sheet_name=0, index_col=0)
data = data.reset_index()
data['Start time 1'] = 0
data['Finish time 1'] = 0
data['Máy trải'] = 0
def schedule_jobs(df, machines):
    # Sắp xếp công việc theo thời gian tới hạn (due date) tăng dần
    sorted_jobs = data.sort_values(by='P trải', ascending=False).values

    # Khởi tạo hàng đợi ưu tiên (min heap) để theo dõi thời gian hoàn thành trên từng máy
    machines_heap = [(0, machine_id) for machine_id in range(machines)]

    # Lập lịch công việc
    for job in sorted_jobs:
        processing_time, machine_id = heapq.heappop(machines_heap)
        start_time=0
        start_time = max(processing_time,start_time)  # Thời điểm bắt đầu là max(thời gian hoàn thành trước đó, thời gian bắt đầu công việc)
        finish_time = start_time + job[5]  # Thời gian hoàn thành công việc là thời gian bắt đầu + thời gian gia công
        df.at[int(job[0]) - 1, 'Start time 1'] = start_time
        df.at[int(job[0]) - 1, 'Finish time 1'] = finish_time
        df.at[int(job[0]) - 1, 'Máy trải'] = machine_id + 1
        heapq.heappush(machines_heap, (finish_time, machine_id))  # Cập nhật hàng đợi với thời gian hoàn thành mới

        #print(f"Công việc {int(job[0])} được lập lịch cho máy {machine_id + 1} từ {start_time} đến {finish_time}")
# Số lượng máy
số_máy_trải= 8

schedule_jobs(data, số_máy_trải)
data_stage2= data.sort_values(by='Máy trải')
display(data_stage2)
#Gán việc cho máy cắt
cutting_machine = []
for ind in data_stage2.index:
    cutting_machine.append((data_stage2['Máy trải'][ind] + 1) // 2)

data_stage2['Máy cắt'] = cutting_machine
data_stage2 = data_stage2.sort_values(by=['Finish time 1', 'Máy cắt'])
display(data_stage2)
#Tính thời gian cho trạm cắt
num_cutting_machine = 4
cur_cutting_time: list[float] = [data_stage2[data_stage2['Máy cắt'] == idx + 1].iloc[0, 8] for idx in range(num_cutting_machine)]

data_stage2['Start time 2'] = 0.0
data_stage2['Finish time 2'] = 0.0

for ind in data_stage2.index:
    cutting_idx = data_stage2.at[ind, 'Máy cắt'] - 1
    data_stage2.at[ind, 'Start time 2'] = max(cur_cutting_time[cutting_idx], data_stage2.at[ind, 'Finish time 1'])
    data_stage2.at[ind, 'Finish time 2'] = data_stage2['Start time 2'][ind] + data_stage2['P cắt'][ind]

    # Update current cutting time
    cur_cutting_time[cutting_idx] = data_stage2.at[ind, 'Finish time 2']
#Tính độ trễ
data_stage2['Độ trễ (phút)']= np.maximum(0, data_stage2['Finish time 2'] - (data_stage2['Due date (h)'] * 60))
print('BẢNG KẾT QUẢ ĐIỀU ĐỘ TRẠM MÁY CẮT')
display(data_stage2)

#Tính hàm mục tiêu
column_count = 'Độ trễ (phút)'
num_tardiness = (data_stage2[column_count] > 0).sum()
print('Số công việc bị trễ theo luật LPT = ', num_tardiness)
sum_tardiness = data_stage2['Độ trễ (phút)'].sum() 
print('Hàm mục tiêu theo luật LPT = ',sum_tardiness,'phút')

# Thứ tự điều độ tại từng trạm
data_stage2
job_order_start_time_1 = data_stage2.sort_values(by='Start time 1')
print("\nKết quả thứ tự điều độ trên trạm trải:")
display(job_order_start_time_1[['Jobs', 'Start time 1']])

job_order_start_time_2 = data_stage2.sort_values(by='Start time 2')
print("\nKết quả thứ tự điều độ trên trạm cắt:")
display(job_order_start_time_2[['Jobs', 'Start time 2']])

,Jobs,Stockout Num,Total rolls,Total yards,Due date (h),P trải,P cắt,Start time 1,Finish time 1,Máy trải
14,15,20230720024151,5,144,9,19.20,24.0,114.14,133.34,1
7,8,20230720024144,10,856,7,114.14,48.0,0.00,114.14,1
15,16,20230720024152,8,251,2,33.47,38.4,96.00,129.47,2
18,19,20230720024155,7,720,9,96.00,33.6,0.00,96.00,2
9,10,20230720024146,10,698,2,93.07,48.0,0.00,93.07,3
13,14,20230720024150,5,275,2,36.67,24.0,93.07,129.74,3
11,12,20230720024148,6,315,4,42.00,28.8,86.14,128.14,4
0,1,20230720024137,10,646,4,86.14,48.0,0.00,86.14,4
8,9,20230720024145,5,113,2,15.07,24.0,122.54,137.61,5
17,18,20230720024154,9,332,4,44.27,43.2,78.27,122.54,5


,Jobs,Stockout Num,Total rolls,Total yards,Due date (h),P trải,P cắt,Start time 1,Finish time 1,Máy trải,Máy cắt
4,5,20230720024141,7,406,2,54.14,33.6,0.00,54.14,8,4
2,3,20230720024139,6,486,4,64.80,28.8,0.00,64.80,7,4
19,20,20230720024156,9,543,2,72.40,43.2,0.00,72.40,6,3
6,7,20230720024143,8,587,2,78.27,38.4,0.00,78.27,5,3
0,1,20230720024137,10,646,4,86.14,48.0,0.00,86.14,4,2
9,10,20230720024146,10,698,2,93.07,48.0,0.00,93.07,3,2
18,19,20230720024155,7,720,9,96.00,33.6,0.00,96.00,2,1
5,6,20230720024142,5,354,7,47.20,24.0,54.14,101.34,8,4
1,2,20230720024138,7,354,2,47.20,33.6,64.80,112.00,7,4
7,8,20230720024144,10,856,7,114.14,48.0,0.00,114.14,1,1


BẢNG KẾT QUẢ ĐIỀU ĐỘ TRẠM MÁY CẮT


,Jobs,Stockout Num,Total rolls,Total yards,Due date (h),P trải,P cắt,Start time 1,Finish time 1,Máy trải,Máy cắt,Start time 2,Finish time 2,Độ trễ (phút)
4,5,20230720024141,7,406,2,54.14,33.6,0.00,54.14,8,4,54.14,87.74,0.00
2,3,20230720024139,6,486,4,64.80,28.8,0.00,64.80,7,4,87.74,116.54,0.00
19,20,20230720024156,9,543,2,72.40,43.2,0.00,72.40,6,3,72.40,115.60,0.00
6,7,20230720024143,8,587,2,78.27,38.4,0.00,78.27,5,3,115.60,154.00,34.00
0,1,20230720024137,10,646,4,86.14,48.0,0.00,86.14,4,2,86.14,134.14,0.00
9,10,20230720024146,10,698,2,93.07,48.0,0.00,93.07,3,2,134.14,182.14,62.14
18,19,20230720024155,7,720,9,96.00,33.6,0.00,96.00,2,1,96.00,129.60,0.00
5,6,20230720024142,5,354,7,47.20,24.0,54.14,101.34,8,4,116.54,140.54,0.00
1,2,20230720024138,7,354,2,47.20,33.6,64.80,112.00,7,4,140.54,174.14,54.14
7,8,20230720024144,10,856,7,114.14,48.0,0.00,114.14,1,1,129.60,177.60,0.00


Số công việc bị trễ theo luật LPT =  7
Hàm mục tiêu theo luật LPT =  568.4200000000001 phút

Kết quả thứ tự điều độ trên trạm trải:


,Jobs,Start time 1
4,5,0.00
18,19,0.00
9,10,0.00
0,1,0.00
7,8,0.00
19,20,0.00
2,3,0.00
6,7,0.00
5,6,54.14
1,2,64.80



Kết quả thứ tự điều độ trên trạm cắt:


,Jobs,Start time 2
4,5,54.14
19,20,72.40
0,1,86.14
2,3,87.74
18,19,96.00
6,7,115.60
5,6,116.54
7,8,129.60
9,10,134.14
1,2,140.54


PHƯƠNG PHÁP LẤY NGẪU NHIÊN

In [53]:
import pandas as pd
import numpy as np
import heapq
import os

# Define the number of random runs
num_random_runs = 6

# Initialize a list to store the results for each random run
all_results = []

for i in range(num_random_runs):
    data = pd.read_excel(os.path.join(data_path, 'Data.xlsx'), sheet_name=0, index_col=0)
    data = data.reset_index()
    data['Start time 1'] = 0
    data['Finish time 1'] = 0
    data['Máy trải'] = 0

    def schedule_jobs(df, machines):
        # Sắp xếp công việc theo pp ngẫu nhiên
        randomly_sorted_jobs = data.sample(frac=1).values

        # Khởi tạo hàng đợi ưu tiên (min heap) để theo dõi thời gian hoàn thành trên từng máy
        machines_heap = [(0, machine_id) for machine_id in range(machines)]

        # Lập lịch công việc
        for job in randomly_sorted_jobs:
            processing_time, machine_id = heapq.heappop(machines_heap)
            start_time = 0
            start_time = max(processing_time, start_time)
            finish_time = start_time + job[5]
            df.at[int(job[0]) - 1, 'Start time 1'] = start_time
            df.at[int(job[0]) - 1, 'Finish time 1'] = finish_time
            df.at[int(job[0]) - 1, 'Máy trải'] = machine_id + 1
            heapq.heappush(machines_heap, (finish_time, machine_id))

    # Số lượng máy
    số_máy_trải = 8
    schedule_jobs(data, số_máy_trải)
    data_stage2 = data.sort_values(by='Máy trải')

    # Gán việc cho máy cắt
    cutting_machine = []
    for ind in data_stage2.index:
        cutting_machine.append((data_stage2['Máy trải'][ind] + 1) // 2)

    data_stage2['Máy cắt'] = cutting_machine
    data_stage2 = data_stage2.sort_values(by=['Finish time 1', 'Máy cắt'])

    # Tính thời gian cho trạm cắt
    num_cutting_machine = 4
    cur_cutting_time = [data_stage2[data_stage2['Máy cắt'] == idx + 1].iloc[0, 8] for idx in range(num_cutting_machine)]

    data_stage2['Start time 2'] = 0.0
    data_stage2['Finish time 2'] = 0.0

    for ind in data_stage2.index:
        cutting_idx = data_stage2.at[ind, 'Máy cắt'] - 1
        data_stage2.at[ind, 'Start time 2'] = max(cur_cutting_time[cutting_idx], data_stage2.at[ind, 'Finish time 1'])
        data_stage2.at[ind, 'Finish time 2'] = data_stage2['Start time 2'][ind] + data_stage2['P cắt'][ind]

        # Update current cutting time
        cur_cutting_time[cutting_idx] = data_stage2.at[ind, 'Finish time 2']

    # Tính độ trễ
    data_stage2['Độ trễ (phút)'] = np.maximum(0, data_stage2['Finish time 2'] - (data_stage2['Due date (h)'] * 60))

    all_results.append(data_stage2.copy())

    # Calculate the objective function for the current random run
    column_count = 'Độ trễ (phút)'
    num_tardiness = (data_stage2[column_count] > 0).sum()
    sum_tardiness = data_stage2['Độ trễ (phút)'].sum()
    print(f'Kết quả lần {i+1}:')
    print('Số công việc bị trễ theo phương pháp ngẫu nhiên = ', num_tardiness)
    print('Hàm mục tiêu theo phương pháp ngẫu nhiên = ', sum_tardiness, 'phút')

    # Print the job order based on 'Start time 1' and 'Start time 2' for the current random run
    job_order_start_time_1 = data_stage2.sort_values(by='Start time 1')
    print("\nThứ tự điều độ trên trạm trải:")
    display(job_order_start_time_1[['Jobs', 'Start time 1']])

    job_order_start_time_2 = data_stage2.sort_values(by='Start time 2')
    print("\nThứ tự điều độ trên trạm cắt:")
    display(job_order_start_time_2[['Jobs', 'Start time 2']])


Kết quả lần 1:
Số công việc bị trễ theo phương pháp ngẫu nhiên =  5
Hàm mục tiêu theo phương pháp ngẫu nhiên =  365.23 phút

Thứ tự điều độ trên trạm trải:


,Jobs,Start time 1
14,15,0.00
18,19,0.00
9,10,0.00
4,5,0.00
0,1,0.00
3,4,0.00
13,14,0.00
12,13,0.00
5,6,19.20
11,12,26.14



Thứ tự điều độ trên trạm cắt:


,Jobs,Start time 2
14,15,19.20
12,13,26.14
13,14,36.67
3,4,44.94
4,5,54.14
8,9,60.67
11,12,68.14
16,17,84.67
5,6,87.74
18,19,96.00


Kết quả lần 2:
Số công việc bị trễ theo phương pháp ngẫu nhiên =  7
Hàm mục tiêu theo phương pháp ngẫu nhiên =  370.97 phút

Thứ tự điều độ trên trạm trải:


...

,Jobs,Start time 2
11,12,42.00
17,18,44.27
3,4,44.94
5,6,47.20
1,2,70.80
19,20,72.40
6,7,78.27
13,14,87.47
10,11,104.40
9,10,111.47


Kết quả lần 6:
Số công việc bị trễ theo phương pháp ngẫu nhiên =  6
Hàm mục tiêu theo phương pháp ngẫu nhiên =  251.22000000000006 phút

Thứ tự điều độ trên trạm trải:


,Jobs,Start time 1
14,15,0.00
6,7,0.00
4,5,0.00
5,6,0.00
0,1,0.00
17,18,0.00
15,16,0.00
3,4,0.00
2,3,19.20
19,20,33.47



Thứ tự điều độ trên trạm cắt:


,Jobs,Start time 2
14,15,19.20
15,16,33.47
3,4,44.94
5,6,47.20
4,5,54.14
10,11,71.20
17,18,71.87
2,3,84.00
0,1,87.74
6,7,95.20


In [16]:
import pandas as pd
import numpy as np
import heapq
import os
data = pd.read_excel(os.path.join(data_path, 'Child 1.xlsx'), sheet_name=0, index_col=0)
data = data.reset_index()
data['Start time 1'] = 0
data['Finish time 1'] = 0
data['Máy trải'] = 0
def schedule_jobs(df, machines):
    # Sắp xếp công việc theo thời gian tới hạn (due date) tăng dần
    sorted_jobs = data.sort_values(by='Jobs').values

    # Khởi tạo hàng đợi ưu tiên (min heap) để theo dõi thời gian hoàn thành trên từng máy
    machines_heap = [(0, machine_id) for machine_id in range(machines)]

    # Lập lịch công việc
    for job in sorted_jobs:
        processing_time, machine_id = heapq.heappop(machines_heap)
        start_time=0
        start_time = max(processing_time,start_time)  # Thời điểm bắt đầu là max(thời gian hoàn thành trước đó, thời gian bắt đầu công việc)
        finish_time = start_time + job[5]  # Thời gian hoàn thành công việc là thời gian bắt đầu + thời gian gia công
        df.at[int(job[0]) - 1, 'Start time 1'] = start_time
        df.at[int(job[0]) - 1, 'Finish time 1'] = finish_time
        df.at[int(job[0]) - 1, 'Máy trải'] = machine_id + 1
        heapq.heappush(machines_heap, (finish_time, machine_id))  # Cập nhật hàng đợi với thời gian hoàn thành mới

        #print(f"Công việc {int(job[0])} được lập lịch cho máy {machine_id + 1} từ {start_time} đến {finish_time}")
# Số lượng máy
số_máy_trải= 8

schedule_jobs(data, số_máy_trải)
data_stage2= data.sort_values(by='Máy trải')
display(data_stage2)
#Gán việc cho máy cắt
cutting_machine = []
for ind in data_stage2.index:
    cutting_machine.append((data_stage2['Máy trải'][ind] + 1) // 2)

data_stage2['Máy cắt'] = cutting_machine
data_stage2 = data_stage2.sort_values(by=['Finish time 1', 'Máy cắt'])
display(data_stage2)
#Tính thời gian cho trạm cắt
num_cutting_machine = 4
cur_cutting_time: list[float] = [data_stage2[data_stage2['Máy cắt'] == idx + 1].iloc[0, 8] for idx in range(num_cutting_machine)]

data_stage2['Start time 2'] = 0.0
data_stage2['Finish time 2'] = 0.0

for ind in data_stage2.index:
    cutting_idx = data_stage2.at[ind, 'Máy cắt'] - 1
    data_stage2.at[ind, 'Start time 2'] = max(cur_cutting_time[cutting_idx], data_stage2.at[ind, 'Finish time 1'])
    data_stage2.at[ind, 'Finish time 2'] = data_stage2['Start time 2'][ind] + data_stage2['P cắt'][ind]

    # Update current cutting time
    cur_cutting_time[cutting_idx] = data_stage2.at[ind, 'Finish time 2']
#Tính độ trễ
data_stage2['Độ trễ (phút)']= np.maximum(0, data_stage2['Finish time 2'] - (data_stage2['Due date (h)'] * 60))
print('BẢNG KẾT QUẢ ĐIỀU ĐỘ TRẠM MÁY CẮT')
display(data_stage2)

#Tính hàm mục tiêu
column_count = 'Độ trễ (phút)'
num_tardiness = (data_stage2[column_count] > 0).sum()
print('Số công việc bị trễ theo luật FCFS = ', num_tardiness)
sum_tardiness = data_stage2['Độ trễ (phút)'].sum() 
print('Hàm mục tiêu theo luật FCFS = ',sum_tardiness,'phút')

# Thứ tự điều độ tại từng trạm
data_stage2
job_order_start_time_1 = data_stage2.sort_values(by='Start time 1')
print("\nKết quả thứ tự điều độ trên trạm trải:")
display(job_order_start_time_1[['Jobs', 'Start time 1']])

job_order_start_time_2 = data_stage2.sort_values(by='Start time 2')
print("\nKết quả thứ tự điều độ trên trạm cắt:")
display(job_order_start_time_2[['Jobs', 'Start time 2']])

,Jobs,Stockout Num,Total rolls,Total yards,Due date (h),P trải,P cắt,Start time 1,Finish time 1,Máy trải
0,11,20230720024147,5,140,7,18.67,24.0,0.00,86.14,1
17,13,20230720024149,6,196,9,26.14,28.8,86.14,130.41,1
9,12,20230720024148,6,315,4,42.00,28.8,47.20,140.27,2
1,9,20230720024145,5,113,2,15.07,24.0,0.00,47.20,2
13,8,20230720024144,10,856,7,114.14,48.0,64.80,101.47,3
2,16,20230720024152,8,251,2,33.47,38.4,0.00,64.80,3
3,14,20230720024150,5,275,2,36.67,24.0,0.00,44.94,4
12,1,20230720024137,10,646,4,86.14,48.0,60.01,86.15,4
8,2,20230720024138,7,354,2,47.20,33.6,44.94,60.01,4
18,15,20230720024151,5,144,9,19.20,24.0,86.15,182.15,4


,Jobs,Stockout Num,Total rolls,Total yards,Due date (h),P trải,P cắt,Start time 1,Finish time 1,Máy trải,Máy cắt
3,14,20230720024150,5,275,2,36.67,24.0,0.00,44.94,4,2
1,9,20230720024145,5,113,2,15.07,24.0,0.00,47.20,2,1
5,20,20230720024156,9,543,2,72.40,43.2,0.00,47.20,6,3
4,4,20230720024140,5,337,2,44.94,24.0,0.00,54.14,5,3
8,2,20230720024138,7,354,2,47.20,33.6,44.94,60.01,4,2
2,16,20230720024152,8,251,2,33.47,38.4,0.00,64.80,3,2
10,3,20230720024139,6,486,4,64.80,28.8,47.20,65.87,6,3
6,7,20230720024143,8,587,2,78.27,38.4,0.00,78.27,7,4
14,6,20230720024142,5,354,7,47.20,24.0,65.87,85.07,6,3
0,11,20230720024147,5,140,7,18.67,24.0,0.00,86.14,1,1


BẢNG KẾT QUẢ ĐIỀU ĐỘ TRẠM MÁY CẮT


,Jobs,Stockout Num,Total rolls,Total yards,Due date (h),P trải,P cắt,Start time 1,Finish time 1,Máy trải,Máy cắt,Start time 2,Finish time 2,Độ trễ (phút)
3,14,20230720024150,5,275,2,36.67,24.0,0.00,44.94,4,2,44.94,68.94,0.00
1,9,20230720024145,5,113,2,15.07,24.0,0.00,47.20,2,1,47.20,71.20,0.00
5,20,20230720024156,9,543,2,72.40,43.2,0.00,47.20,6,3,47.20,90.40,0.00
4,4,20230720024140,5,337,2,44.94,24.0,0.00,54.14,5,3,90.40,114.40,0.00
8,2,20230720024138,7,354,2,47.20,33.6,44.94,60.01,4,2,68.94,102.54,0.00
2,16,20230720024152,8,251,2,33.47,38.4,0.00,64.80,3,2,102.54,140.94,20.94
10,3,20230720024139,6,486,4,64.80,28.8,47.20,65.87,6,3,114.40,143.20,0.00
6,7,20230720024143,8,587,2,78.27,38.4,0.00,78.27,7,4,78.27,116.67,0.00
14,6,20230720024142,5,354,7,47.20,24.0,65.87,85.07,6,3,143.20,167.20,0.00
0,11,20230720024147,5,140,7,18.67,24.0,0.00,86.14,1,1,86.14,110.14,0.00


Số công việc bị trễ theo luật FCFS =  3
Hàm mục tiêu theo luật FCFS =  129.47999999999996 phút

Kết quả thứ tự điều độ trên trạm trải:


,Jobs,Start time 1
3,14,0.00
7,10,0.00
6,7,0.00
2,16,0.00
0,11,0.00
4,4,0.00
5,20,0.00
1,9,0.00
8,2,44.94
10,3,47.20



Kết quả thứ tự điều độ trên trạm cắt:


,Jobs,Start time 2
3,14,44.94
1,9,47.20
5,20,47.20
8,2,68.94
6,7,78.27
0,11,86.14
4,4,90.40
2,16,102.54
10,3,114.40
15,5,116.67


In [17]:
import pandas as pd
import numpy as np
import heapq
import os
data = pd.read_excel(os.path.join(data_path, 'Child 2.xlsx'), sheet_name=0, index_col=0)
data = data.reset_index()
data['Start time 1'] = 0
data['Finish time 1'] = 0
data['Máy trải'] = 0
def schedule_jobs(df, machines):
    # Sắp xếp công việc theo thời gian tới hạn (due date) tăng dần
    sorted_jobs = data.sort_values(by='Jobs').values

    # Khởi tạo hàng đợi ưu tiên (min heap) để theo dõi thời gian hoàn thành trên từng máy
    machines_heap = [(0, machine_id) for machine_id in range(machines)]

    # Lập lịch công việc
    for job in sorted_jobs:
        processing_time, machine_id = heapq.heappop(machines_heap)
        start_time=0
        start_time = max(processing_time,start_time)  # Thời điểm bắt đầu là max(thời gian hoàn thành trước đó, thời gian bắt đầu công việc)
        finish_time = start_time + job[5]  # Thời gian hoàn thành công việc là thời gian bắt đầu + thời gian gia công
        df.at[int(job[0]) - 1, 'Start time 1'] = start_time
        df.at[int(job[0]) - 1, 'Finish time 1'] = finish_time
        df.at[int(job[0]) - 1, 'Máy trải'] = machine_id + 1
        heapq.heappush(machines_heap, (finish_time, machine_id))  # Cập nhật hàng đợi với thời gian hoàn thành mới

        #print(f"Công việc {int(job[0])} được lập lịch cho máy {machine_id + 1} từ {start_time} đến {finish_time}")
# Số lượng máy
số_máy_trải= 8

schedule_jobs(data, số_máy_trải)
data_stage2= data.sort_values(by='Máy trải')
display(data_stage2)
#Gán việc cho máy cắt
cutting_machine = []
for ind in data_stage2.index:
    cutting_machine.append((data_stage2['Máy trải'][ind] + 1) // 2)

data_stage2['Máy cắt'] = cutting_machine
data_stage2 = data_stage2.sort_values(by=['Finish time 1', 'Máy cắt'])
display(data_stage2)
#Tính thời gian cho trạm cắt
num_cutting_machine = 4
cur_cutting_time: list[float] = [data_stage2[data_stage2['Máy cắt'] == idx + 1].iloc[0, 8] for idx in range(num_cutting_machine)]

data_stage2['Start time 2'] = 0.0
data_stage2['Finish time 2'] = 0.0

for ind in data_stage2.index:
    cutting_idx = data_stage2.at[ind, 'Máy cắt'] - 1
    data_stage2.at[ind, 'Start time 2'] = max(cur_cutting_time[cutting_idx], data_stage2.at[ind, 'Finish time 1'])
    data_stage2.at[ind, 'Finish time 2'] = data_stage2['Start time 2'][ind] + data_stage2['P cắt'][ind]

    # Update current cutting time
    cur_cutting_time[cutting_idx] = data_stage2.at[ind, 'Finish time 2']
#Tính độ trễ
data_stage2['Độ trễ (phút)']= np.maximum(0, data_stage2['Finish time 2'] - (data_stage2['Due date (h)'] * 60))
print('BẢNG KẾT QUẢ ĐIỀU ĐỘ TRẠM MÁY CẮT')
display(data_stage2)

#Tính hàm mục tiêu
column_count = 'Độ trễ (phút)'
num_tardiness = (data_stage2[column_count] > 0).sum()
print('Số công việc bị trễ theo luật FCFS = ', num_tardiness)
sum_tardiness = data_stage2['Độ trễ (phút)'].sum() 
print('Hàm mục tiêu theo luật FCFS = ',sum_tardiness,'phút')

# Thứ tự điều độ tại từng trạm
data_stage2
job_order_start_time_1 = data_stage2.sort_values(by='Start time 1')
print("\nKết quả thứ tự điều độ trên trạm trải:")
display(job_order_start_time_1[['Jobs', 'Start time 1']])

job_order_start_time_2 = data_stage2.sort_values(by='Start time 2')
print("\nKết quả thứ tự điều độ trên trạm cắt:")
display(job_order_start_time_2[['Jobs', 'Start time 2']])

,Jobs,Stockout Num,Total rolls,Total yards,Due date (h),P trải,P cắt,Start time 1,Finish time 1,Máy trải
0,15,20230720024151,5,144,9,19.20,24.0,0.00,86.14,1
17,10,20230720024146,10,698,2,93.07,48.0,86.14,130.41,1
9,3,20230720024139,6,486,4,64.80,28.8,47.20,140.27,2
1,12,20230720024148,6,315,4,42.00,28.8,0.00,47.20,2
13,14,20230720024150,5,275,2,36.67,24.0,64.80,101.47,3
2,4,20230720024140,5,337,2,44.94,24.0,0.00,64.80,3
3,6,20230720024142,5,354,7,47.20,24.0,0.00,44.94,4
12,16,20230720024152,8,251,2,33.47,38.4,60.01,86.15,4
8,17,20230720024153,7,225,9,30.00,33.6,44.94,60.01,4
18,9,20230720024145,5,113,2,15.07,24.0,86.15,182.15,4


,Jobs,Stockout Num,Total rolls,Total yards,Due date (h),P trải,P cắt,Start time 1,Finish time 1,Máy trải,Máy cắt
3,6,20230720024142,5,354,7,47.20,24.0,0.00,44.94,4,2
1,12,20230720024148,6,315,4,42.00,28.8,0.00,47.20,2,1
5,7,20230720024143,8,587,2,78.27,38.4,0.00,47.20,6,3
4,5,20230720024141,7,406,2,54.14,33.6,0.00,54.14,5,3
8,17,20230720024153,7,225,9,30.00,33.6,44.94,60.01,4,2
2,4,20230720024140,5,337,2,44.94,24.0,0.00,64.80,3,2
10,20,20230720024156,9,543,2,72.40,43.2,47.20,65.87,6,3
6,1,20230720024137,10,646,4,86.14,48.0,0.00,78.27,7,4
14,11,20230720024147,5,140,7,18.67,24.0,65.87,85.07,6,3
0,15,20230720024151,5,144,9,19.20,24.0,0.00,86.14,1,1


BẢNG KẾT QUẢ ĐIỀU ĐỘ TRẠM MÁY CẮT


,Jobs,Stockout Num,Total rolls,Total yards,Due date (h),P trải,P cắt,Start time 1,Finish time 1,Máy trải,Máy cắt,Start time 2,Finish time 2,Độ trễ (phút)
3,6,20230720024142,5,354,7,47.20,24.0,0.00,44.94,4,2,44.94,68.94,0.00
1,12,20230720024148,6,315,4,42.00,28.8,0.00,47.20,2,1,47.20,76.00,0.00
5,7,20230720024143,8,587,2,78.27,38.4,0.00,47.20,6,3,47.20,85.60,0.00
4,5,20230720024141,7,406,2,54.14,33.6,0.00,54.14,5,3,85.60,119.20,0.00
8,17,20230720024153,7,225,9,30.00,33.6,44.94,60.01,4,2,68.94,102.54,0.00
2,4,20230720024140,5,337,2,44.94,24.0,0.00,64.80,3,2,102.54,126.54,6.54
10,20,20230720024156,9,543,2,72.40,43.2,47.20,65.87,6,3,119.20,162.40,42.40
6,1,20230720024137,10,646,4,86.14,48.0,0.00,78.27,7,4,78.27,126.27,0.00
14,11,20230720024147,5,140,7,18.67,24.0,65.87,85.07,6,3,162.40,186.40,0.00
0,15,20230720024151,5,144,9,19.20,24.0,0.00,86.14,1,1,86.14,110.14,0.00


Số công việc bị trễ theo luật FCFS =  7
Hàm mục tiêu theo luật FCFS =  354.03999999999996 phút

Kết quả thứ tự điều độ trên trạm trải:


,Jobs,Start time 1
3,6,0.00
7,8,0.00
6,1,0.00
2,4,0.00
0,15,0.00
4,5,0.00
5,7,0.00
1,12,0.00
8,17,44.94
10,20,47.20



Kết quả thứ tự điều độ trên trạm cắt:


,Jobs,Start time 2
3,6,44.94
1,12,47.20
5,7,47.20
8,17,68.94
6,1,78.27
4,5,85.60
0,15,86.14
2,4,102.54
10,20,119.20
15,2,126.27


In [18]:
import pandas as pd
import numpy as np
import heapq
import os
data = pd.read_excel(os.path.join(data_path, 'Child 3.xlsx'), sheet_name=0, index_col=0)
data = data.reset_index()
data['Start time 1'] = 0
data['Finish time 1'] = 0
data['Máy trải'] = 0
def schedule_jobs(df, machines):
    # Sắp xếp công việc theo thời gian tới hạn (due date) tăng dần
    sorted_jobs = data.sort_values(by='Jobs').values

    # Khởi tạo hàng đợi ưu tiên (min heap) để theo dõi thời gian hoàn thành trên từng máy
    machines_heap = [(0, machine_id) for machine_id in range(machines)]

    # Lập lịch công việc
    for job in sorted_jobs:
        processing_time, machine_id = heapq.heappop(machines_heap)
        start_time=0
        start_time = max(processing_time,start_time)  # Thời điểm bắt đầu là max(thời gian hoàn thành trước đó, thời gian bắt đầu công việc)
        finish_time = start_time + job[5]  # Thời gian hoàn thành công việc là thời gian bắt đầu + thời gian gia công
        df.at[int(job[0]) - 1, 'Start time 1'] = start_time
        df.at[int(job[0]) - 1, 'Finish time 1'] = finish_time
        df.at[int(job[0]) - 1, 'Máy trải'] = machine_id + 1
        heapq.heappush(machines_heap, (finish_time, machine_id))  # Cập nhật hàng đợi với thời gian hoàn thành mới

        #print(f"Công việc {int(job[0])} được lập lịch cho máy {machine_id + 1} từ {start_time} đến {finish_time}")
# Số lượng máy
số_máy_trải= 8

schedule_jobs(data, số_máy_trải)
data_stage2= data.sort_values(by='Máy trải')
display(data_stage2)
#Gán việc cho máy cắt
cutting_machine = []
for ind in data_stage2.index:
    cutting_machine.append((data_stage2['Máy trải'][ind] + 1) // 2)

data_stage2['Máy cắt'] = cutting_machine
data_stage2 = data_stage2.sort_values(by=['Finish time 1', 'Máy cắt'])
display(data_stage2)
#Tính thời gian cho trạm cắt
num_cutting_machine = 4
cur_cutting_time: list[float] = [data_stage2[data_stage2['Máy cắt'] == idx + 1].iloc[0, 8] for idx in range(num_cutting_machine)]

data_stage2['Start time 2'] = 0.0
data_stage2['Finish time 2'] = 0.0

for ind in data_stage2.index:
    cutting_idx = data_stage2.at[ind, 'Máy cắt'] - 1
    data_stage2.at[ind, 'Start time 2'] = max(cur_cutting_time[cutting_idx], data_stage2.at[ind, 'Finish time 1'])
    data_stage2.at[ind, 'Finish time 2'] = data_stage2['Start time 2'][ind] + data_stage2['P cắt'][ind]

    # Update current cutting time
    cur_cutting_time[cutting_idx] = data_stage2.at[ind, 'Finish time 2']
#Tính độ trễ
data_stage2['Độ trễ (phút)']= np.maximum(0, data_stage2['Finish time 2'] - (data_stage2['Due date (h)'] * 60))
print('BẢNG KẾT QUẢ ĐIỀU ĐỘ TRẠM MÁY CẮT')
display(data_stage2)

#Tính hàm mục tiêu
column_count = 'Độ trễ (phút)'
num_tardiness = (data_stage2[column_count] > 0).sum()
print('Số công việc bị trễ theo luật FCFS = ', num_tardiness)
sum_tardiness = data_stage2['Độ trễ (phút)'].sum() 
print('Hàm mục tiêu theo luật FCFS = ',sum_tardiness,'phút')

# Thứ tự điều độ tại từng trạm
data_stage2
job_order_start_time_1 = data_stage2.sort_values(by='Start time 1')
print("\nKết quả thứ tự điều độ trên trạm trải:")
display(job_order_start_time_1[['Jobs', 'Start time 1']])

job_order_start_time_2 = data_stage2.sort_values(by='Start time 2')
print("\nKết quả thứ tự điều độ trên trạm cắt:")
display(job_order_start_time_2[['Jobs', 'Start time 2']])

,Jobs,Stockout Num,Total rolls,Total yards,Due date (h),P trải,P cắt,Start time 1,Finish time 1,Máy trải
0,12,20230720024148,6,315,4,42.00,28.8,0.00,86.14,1
17,3,20230720024139,6,486,4,64.80,28.8,86.14,130.41,1
9,11,20230720024147,5,140,7,18.67,24.0,47.20,140.27,2
1,18,20230720024154,9,332,4,44.27,43.2,0.00,47.20,2
13,8,20230720024144,10,856,7,114.14,48.0,64.80,101.47,3
2,4,20230720024140,5,337,2,44.94,24.0,0.00,64.80,3
3,6,20230720024142,5,354,7,47.20,24.0,0.00,44.94,4
12,1,20230720024137,10,646,4,86.14,48.0,60.01,86.15,4
8,9,20230720024145,5,113,2,15.07,24.0,44.94,60.01,4
18,17,20230720024153,7,225,9,30.00,33.6,86.15,182.15,4


,Jobs,Stockout Num,Total rolls,Total yards,Due date (h),P trải,P cắt,Start time 1,Finish time 1,Máy trải,Máy cắt
3,6,20230720024142,5,354,7,47.20,24.0,0.00,44.94,4,2
1,18,20230720024154,9,332,4,44.27,43.2,0.00,47.20,2,1
5,20,20230720024156,9,543,2,72.40,43.2,0.00,47.20,6,3
4,2,20230720024138,7,354,2,47.20,33.6,0.00,54.14,5,3
8,9,20230720024145,5,113,2,15.07,24.0,44.94,60.01,4,2
2,4,20230720024140,5,337,2,44.94,24.0,0.00,64.80,3,2
10,14,20230720024150,5,275,2,36.67,24.0,47.20,65.87,6,3
6,7,20230720024143,8,587,2,78.27,38.4,0.00,78.27,7,4
14,15,20230720024151,5,144,9,19.20,24.0,65.87,85.07,6,3
0,12,20230720024148,6,315,4,42.00,28.8,0.00,86.14,1,1


BẢNG KẾT QUẢ ĐIỀU ĐỘ TRẠM MÁY CẮT


,Jobs,Stockout Num,Total rolls,Total yards,Due date (h),P trải,P cắt,Start time 1,Finish time 1,Máy trải,Máy cắt,Start time 2,Finish time 2,Độ trễ (phút)
3,6,20230720024142,5,354,7,47.20,24.0,0.00,44.94,4,2,44.94,68.94,0.00
1,18,20230720024154,9,332,4,44.27,43.2,0.00,47.20,2,1,47.20,90.40,0.00
5,20,20230720024156,9,543,2,72.40,43.2,0.00,47.20,6,3,47.20,90.40,0.00
4,2,20230720024138,7,354,2,47.20,33.6,0.00,54.14,5,3,90.40,124.00,4.00
8,9,20230720024145,5,113,2,15.07,24.0,44.94,60.01,4,2,68.94,92.94,0.00
2,4,20230720024140,5,337,2,44.94,24.0,0.00,64.80,3,2,92.94,116.94,0.00
10,14,20230720024150,5,275,2,36.67,24.0,47.20,65.87,6,3,124.00,148.00,28.00
6,7,20230720024143,8,587,2,78.27,38.4,0.00,78.27,7,4,78.27,116.67,0.00
14,15,20230720024151,5,144,9,19.20,24.0,65.87,85.07,6,3,148.00,172.00,0.00
0,12,20230720024148,6,315,4,42.00,28.8,0.00,86.14,1,1,90.40,119.20,0.00


Số công việc bị trễ theo luật FCFS =  5
Hàm mục tiêu theo luật FCFS =  315.07 phút

Kết quả thứ tự điều độ trên trạm trải:


,Jobs,Start time 1
3,6,0.00
7,10,0.00
6,7,0.00
2,4,0.00
0,12,0.00
4,2,0.00
5,20,0.00
1,18,0.00
8,9,44.94
10,14,47.20



Kết quả thứ tự điều độ trên trạm cắt:


,Jobs,Start time 2
3,6,44.94
1,18,47.20
5,20,47.20
8,9,68.94
6,7,78.27
0,12,90.40
4,2,90.40
2,4,92.94
15,13,116.67
12,1,116.94


In [19]:
import pandas as pd
import numpy as np
import heapq
import os
data = pd.read_excel(os.path.join(data_path, 'Child 4.xlsx'), sheet_name=0, index_col=0)
data = data.reset_index()
data['Start time 1'] = 0
data['Finish time 1'] = 0
data['Máy trải'] = 0
def schedule_jobs(df, machines):
    # Sắp xếp công việc theo thời gian tới hạn (due date) tăng dần
    sorted_jobs = data.sort_values(by='Jobs').values

    # Khởi tạo hàng đợi ưu tiên (min heap) để theo dõi thời gian hoàn thành trên từng máy
    machines_heap = [(0, machine_id) for machine_id in range(machines)]

    # Lập lịch công việc
    for job in sorted_jobs:
        processing_time, machine_id = heapq.heappop(machines_heap)
        start_time=0
        start_time = max(processing_time,start_time)  # Thời điểm bắt đầu là max(thời gian hoàn thành trước đó, thời gian bắt đầu công việc)
        finish_time = start_time + job[5]  # Thời gian hoàn thành công việc là thời gian bắt đầu + thời gian gia công
        df.at[int(job[0]) - 1, 'Start time 1'] = start_time
        df.at[int(job[0]) - 1, 'Finish time 1'] = finish_time
        df.at[int(job[0]) - 1, 'Máy trải'] = machine_id + 1
        heapq.heappush(machines_heap, (finish_time, machine_id))  # Cập nhật hàng đợi với thời gian hoàn thành mới

        #print(f"Công việc {int(job[0])} được lập lịch cho máy {machine_id + 1} từ {start_time} đến {finish_time}")
# Số lượng máy
số_máy_trải= 8

schedule_jobs(data, số_máy_trải)
data_stage2= data.sort_values(by='Máy trải')
display(data_stage2)
#Gán việc cho máy cắt
cutting_machine = []
for ind in data_stage2.index:
    cutting_machine.append((data_stage2['Máy trải'][ind] + 1) // 2)

data_stage2['Máy cắt'] = cutting_machine
data_stage2 = data_stage2.sort_values(by=['Finish time 1', 'Máy cắt'])
display(data_stage2)
#Tính thời gian cho trạm cắt
num_cutting_machine = 4
cur_cutting_time: list[float] = [data_stage2[data_stage2['Máy cắt'] == idx + 1].iloc[0, 8] for idx in range(num_cutting_machine)]

data_stage2['Start time 2'] = 0.0
data_stage2['Finish time 2'] = 0.0

for ind in data_stage2.index:
    cutting_idx = data_stage2.at[ind, 'Máy cắt'] - 1
    data_stage2.at[ind, 'Start time 2'] = max(cur_cutting_time[cutting_idx], data_stage2.at[ind, 'Finish time 1'])
    data_stage2.at[ind, 'Finish time 2'] = data_stage2['Start time 2'][ind] + data_stage2['P cắt'][ind]

    # Update current cutting time
    cur_cutting_time[cutting_idx] = data_stage2.at[ind, 'Finish time 2']
#Tính độ trễ
data_stage2['Độ trễ (phút)']= np.maximum(0, data_stage2['Finish time 2'] - (data_stage2['Due date (h)'] * 60))
print('BẢNG KẾT QUẢ ĐIỀU ĐỘ TRẠM MÁY CẮT')
display(data_stage2)

#Tính hàm mục tiêu
column_count = 'Độ trễ (phút)'
num_tardiness = (data_stage2[column_count] > 0).sum()
print('Số công việc bị trễ theo luật FCFS = ', num_tardiness)
sum_tardiness = data_stage2['Độ trễ (phút)'].sum() 
print('Hàm mục tiêu theo luật FCFS = ',sum_tardiness,'phút')

# Thứ tự điều độ tại từng trạm
data_stage2
job_order_start_time_1 = data_stage2.sort_values(by='Start time 1')
print("\nKết quả thứ tự điều độ trên trạm trải:")
display(job_order_start_time_1[['Jobs', 'Start time 1']])

job_order_start_time_2 = data_stage2.sort_values(by='Start time 2')
print("\nKết quả thứ tự điều độ trên trạm cắt:")
display(job_order_start_time_2[['Jobs', 'Start time 2']])

,Jobs,Stockout Num,Total rolls,Total yards,Due date (h),P trải,P cắt,Start time 1,Finish time 1,Máy trải
0,4,20230720024140,5,337,2,44.94,24.0,0.00,86.14,1
17,17,20230720024153,7,225,9,30.00,33.6,86.14,130.41,1
9,10,20230720024146,10,698,2,93.07,48.0,47.20,140.27,2
1,8,20230720024144,10,856,7,114.14,48.0,0.00,47.20,2
13,15,20230720024151,5,144,9,19.20,24.0,64.80,101.47,3
2,7,20230720024143,8,587,2,78.27,38.4,0.00,64.80,3
3,3,20230720024139,6,486,4,64.80,28.8,0.00,44.94,4
12,14,20230720024150,5,275,2,36.67,24.0,60.01,86.15,4
8,11,20230720024147,5,140,7,18.67,24.0,44.94,60.01,4
18,18,20230720024154,9,332,4,44.27,43.2,86.15,182.15,4


,Jobs,Stockout Num,Total rolls,Total yards,Due date (h),P trải,P cắt,Start time 1,Finish time 1,Máy trải,Máy cắt
3,3,20230720024139,6,486,4,64.80,28.8,0.00,44.94,4,2
1,8,20230720024144,10,856,7,114.14,48.0,0.00,47.20,2,1
5,5,20230720024141,7,406,2,54.14,33.6,0.00,47.20,6,3
4,1,20230720024137,10,646,4,86.14,48.0,0.00,54.14,5,3
8,11,20230720024147,5,140,7,18.67,24.0,44.94,60.01,4,2
2,7,20230720024143,8,587,2,78.27,38.4,0.00,64.80,3,2
10,12,20230720024148,6,315,4,42.00,28.8,47.20,65.87,6,3
6,6,20230720024142,5,354,7,47.20,24.0,0.00,78.27,7,4
14,16,20230720024152,8,251,2,33.47,38.4,65.87,85.07,6,3
0,4,20230720024140,5,337,2,44.94,24.0,0.00,86.14,1,1


BẢNG KẾT QUẢ ĐIỀU ĐỘ TRẠM MÁY CẮT


,Jobs,Stockout Num,Total rolls,Total yards,Due date (h),P trải,P cắt,Start time 1,Finish time 1,Máy trải,Máy cắt,Start time 2,Finish time 2,Độ trễ (phút)
3,3,20230720024139,6,486,4,64.80,28.8,0.00,44.94,4,2,44.94,73.74,0.00
1,8,20230720024144,10,856,7,114.14,48.0,0.00,47.20,2,1,47.20,95.20,0.00
5,5,20230720024141,7,406,2,54.14,33.6,0.00,47.20,6,3,47.20,80.80,0.00
4,1,20230720024137,10,646,4,86.14,48.0,0.00,54.14,5,3,80.80,128.80,0.00
8,11,20230720024147,5,140,7,18.67,24.0,44.94,60.01,4,2,73.74,97.74,0.00
2,7,20230720024143,8,587,2,78.27,38.4,0.00,64.80,3,2,97.74,136.14,16.14
10,12,20230720024148,6,315,4,42.00,28.8,47.20,65.87,6,3,128.80,157.60,0.00
6,6,20230720024142,5,354,7,47.20,24.0,0.00,78.27,7,4,78.27,102.27,0.00
14,16,20230720024152,8,251,2,33.47,38.4,65.87,85.07,6,3,157.60,196.00,76.00
0,4,20230720024140,5,337,2,44.94,24.0,0.00,86.14,1,1,95.20,119.20,0.00


Số công việc bị trễ theo luật FCFS =  7
Hàm mục tiêu theo luật FCFS =  584.0300000000001 phút

Kết quả thứ tự điều độ trên trạm trải:


,Jobs,Start time 1
3,3,0.00
7,2,0.00
6,6,0.00
2,7,0.00
0,4,0.00
4,1,0.00
5,5,0.00
1,8,0.00
8,11,44.94
10,12,47.20



Kết quả thứ tự điều độ trên trạm cắt:


,Jobs,Start time 2
3,3,44.94
1,8,47.20
5,5,47.20
8,11,73.74
6,6,78.27
4,1,80.80
0,4,95.20
2,7,97.74
15,19,111.74
10,12,128.80


In [20]:
import pandas as pd
import numpy as np
import heapq
import os
data = pd.read_excel(os.path.join(data_path, 'Mutation 1.xlsx'), sheet_name=0, index_col=0)
data = data.reset_index()
data['Start time 1'] = 0
data['Finish time 1'] = 0
data['Máy trải'] = 0
def schedule_jobs(df, machines):
    # Sắp xếp công việc theo thời gian tới hạn (due date) tăng dần
    sorted_jobs = data.sort_values(by='Jobs').values

    # Khởi tạo hàng đợi ưu tiên (min heap) để theo dõi thời gian hoàn thành trên từng máy
    machines_heap = [(0, machine_id) for machine_id in range(machines)]

    # Lập lịch công việc
    for job in sorted_jobs:
        processing_time, machine_id = heapq.heappop(machines_heap)
        start_time=0
        start_time = max(processing_time,start_time)  # Thời điểm bắt đầu là max(thời gian hoàn thành trước đó, thời gian bắt đầu công việc)
        finish_time = start_time + job[5]  # Thời gian hoàn thành công việc là thời gian bắt đầu + thời gian gia công
        df.at[int(job[0]) - 1, 'Start time 1'] = start_time
        df.at[int(job[0]) - 1, 'Finish time 1'] = finish_time
        df.at[int(job[0]) - 1, 'Máy trải'] = machine_id + 1
        heapq.heappush(machines_heap, (finish_time, machine_id))  # Cập nhật hàng đợi với thời gian hoàn thành mới

        #print(f"Công việc {int(job[0])} được lập lịch cho máy {machine_id + 1} từ {start_time} đến {finish_time}")
# Số lượng máy
số_máy_trải= 8

schedule_jobs(data, số_máy_trải)
data_stage2= data.sort_values(by='Máy trải')
display(data_stage2)
#Gán việc cho máy cắt
cutting_machine = []
for ind in data_stage2.index:
    cutting_machine.append((data_stage2['Máy trải'][ind] + 1) // 2)

data_stage2['Máy cắt'] = cutting_machine
data_stage2 = data_stage2.sort_values(by=['Finish time 1', 'Máy cắt'])
display(data_stage2)
#Tính thời gian cho trạm cắt
num_cutting_machine = 4
cur_cutting_time: list[float] = [data_stage2[data_stage2['Máy cắt'] == idx + 1].iloc[0, 8] for idx in range(num_cutting_machine)]

data_stage2['Start time 2'] = 0.0
data_stage2['Finish time 2'] = 0.0

for ind in data_stage2.index:
    cutting_idx = data_stage2.at[ind, 'Máy cắt'] - 1
    data_stage2.at[ind, 'Start time 2'] = max(cur_cutting_time[cutting_idx], data_stage2.at[ind, 'Finish time 1'])
    data_stage2.at[ind, 'Finish time 2'] = data_stage2['Start time 2'][ind] + data_stage2['P cắt'][ind]

    # Update current cutting time
    cur_cutting_time[cutting_idx] = data_stage2.at[ind, 'Finish time 2']
#Tính độ trễ
data_stage2['Độ trễ (phút)']= np.maximum(0, data_stage2['Finish time 2'] - (data_stage2['Due date (h)'] * 60))
print('BẢNG KẾT QUẢ ĐIỀU ĐỘ TRẠM MÁY CẮT')
display(data_stage2)

#Tính hàm mục tiêu
column_count = 'Độ trễ (phút)'
num_tardiness = (data_stage2[column_count] > 0).sum()
print('Số công việc bị trễ theo luật FCFS = ', num_tardiness)
sum_tardiness = data_stage2['Độ trễ (phút)'].sum() 
print('Hàm mục tiêu theo luật FCFS = ',sum_tardiness,'phút')

# Thứ tự điều độ tại từng trạm
data_stage2
job_order_start_time_1 = data_stage2.sort_values(by='Start time 1')
print("\nKết quả thứ tự điều độ trên trạm trải:")
display(job_order_start_time_1[['Jobs', 'Start time 1']])

job_order_start_time_2 = data_stage2.sort_values(by='Start time 2')
print("\nKết quả thứ tự điều độ trên trạm cắt:")
display(job_order_start_time_2[['Jobs', 'Start time 2']])

,Jobs,Stockout Num,Total rolls,Total yards,Due date (h),P trải,P cắt,Start time 1,Finish time 1,Máy trải
0,9,20230720024145,5,113,2,15.07,24.0,0.00,86.14,1
17,13,20230720024149,6,196,9,26.14,28.8,86.14,130.41,1
9,12,20230720024148,6,315,4,42.00,28.8,47.20,140.27,2
1,16,20230720024152,8,251,2,33.47,38.4,0.00,47.20,2
13,8,20230720024144,10,856,7,114.14,48.0,64.80,101.47,3
2,14,20230720024150,5,275,2,36.67,24.0,0.00,64.80,3
3,4,20230720024140,5,337,2,44.94,24.0,0.00,44.94,4
12,1,20230720024137,10,646,4,86.14,48.0,60.01,86.15,4
8,2,20230720024138,7,354,2,47.20,33.6,44.94,60.01,4
18,15,20230720024151,5,144,9,19.20,24.0,86.15,182.15,4


,Jobs,Stockout Num,Total rolls,Total yards,Due date (h),P trải,P cắt,Start time 1,Finish time 1,Máy trải,Máy cắt
3,4,20230720024140,5,337,2,44.94,24.0,0.00,44.94,4,2
1,16,20230720024152,8,251,2,33.47,38.4,0.00,47.20,2,1
5,20,20230720024156,9,543,2,72.40,43.2,0.00,47.20,6,3
4,5,20230720024141,7,406,2,54.14,33.6,0.00,54.14,5,3
8,2,20230720024138,7,354,2,47.20,33.6,44.94,60.01,4,2
2,14,20230720024150,5,275,2,36.67,24.0,0.00,64.80,3,2
10,3,20230720024139,6,486,4,64.80,28.8,47.20,65.87,6,3
6,10,20230720024146,10,698,2,93.07,48.0,0.00,78.27,7,4
14,11,20230720024147,5,140,7,18.67,24.0,65.87,85.07,6,3
0,9,20230720024145,5,113,2,15.07,24.0,0.00,86.14,1,1


BẢNG KẾT QUẢ ĐIỀU ĐỘ TRẠM MÁY CẮT


,Jobs,Stockout Num,Total rolls,Total yards,Due date (h),P trải,P cắt,Start time 1,Finish time 1,Máy trải,Máy cắt,Start time 2,Finish time 2,Độ trễ (phút)
3,4,20230720024140,5,337,2,44.94,24.0,0.00,44.94,4,2,44.94,68.94,0.00
1,16,20230720024152,8,251,2,33.47,38.4,0.00,47.20,2,1,47.20,85.60,0.00
5,20,20230720024156,9,543,2,72.40,43.2,0.00,47.20,6,3,47.20,90.40,0.00
4,5,20230720024141,7,406,2,54.14,33.6,0.00,54.14,5,3,90.40,124.00,4.00
8,2,20230720024138,7,354,2,47.20,33.6,44.94,60.01,4,2,68.94,102.54,0.00
2,14,20230720024150,5,275,2,36.67,24.0,0.00,64.80,3,2,102.54,126.54,6.54
10,3,20230720024139,6,486,4,64.80,28.8,47.20,65.87,6,3,124.00,152.80,0.00
6,10,20230720024146,10,698,2,93.07,48.0,0.00,78.27,7,4,78.27,126.27,6.27
14,11,20230720024147,5,140,7,18.67,24.0,65.87,85.07,6,3,152.80,176.80,0.00
0,9,20230720024145,5,113,2,15.07,24.0,0.00,86.14,1,1,86.14,110.14,0.00


Số công việc bị trễ theo luật FCFS =  4
Hàm mục tiêu theo luật FCFS =  85.47999999999998 phút

Kết quả thứ tự điều độ trên trạm trải:


,Jobs,Start time 1
3,4,0.00
7,7,0.00
6,10,0.00
2,14,0.00
0,9,0.00
4,5,0.00
5,20,0.00
1,16,0.00
8,2,44.94
10,3,47.20



Kết quả thứ tự điều độ trên trạm cắt:


,Jobs,Start time 2
3,4,44.94
1,16,47.20
5,20,47.20
8,2,68.94
6,10,78.27
0,9,86.14
4,5,90.40
2,14,102.54
10,3,124.00
15,6,126.27


In [21]:
import pandas as pd
import numpy as np
import heapq
import os
data = pd.read_excel(os.path.join(data_path, 'Mutation 2.xlsx'), sheet_name=0, index_col=0)
data = data.reset_index()
data['Start time 1'] = 0
data['Finish time 1'] = 0
data['Máy trải'] = 0
def schedule_jobs(df, machines):
    # Sắp xếp công việc theo thời gian tới hạn (due date) tăng dần
    sorted_jobs = data.sort_values(by='Jobs').values

    # Khởi tạo hàng đợi ưu tiên (min heap) để theo dõi thời gian hoàn thành trên từng máy
    machines_heap = [(0, machine_id) for machine_id in range(machines)]

    # Lập lịch công việc
    for job in sorted_jobs:
        processing_time, machine_id = heapq.heappop(machines_heap)
        start_time=0
        start_time = max(processing_time,start_time)  # Thời điểm bắt đầu là max(thời gian hoàn thành trước đó, thời gian bắt đầu công việc)
        finish_time = start_time + job[5]  # Thời gian hoàn thành công việc là thời gian bắt đầu + thời gian gia công
        df.at[int(job[0]) - 1, 'Start time 1'] = start_time
        df.at[int(job[0]) - 1, 'Finish time 1'] = finish_time
        df.at[int(job[0]) - 1, 'Máy trải'] = machine_id + 1
        heapq.heappush(machines_heap, (finish_time, machine_id))  # Cập nhật hàng đợi với thời gian hoàn thành mới

        #print(f"Công việc {int(job[0])} được lập lịch cho máy {machine_id + 1} từ {start_time} đến {finish_time}")
# Số lượng máy
số_máy_trải= 8

schedule_jobs(data, số_máy_trải)
data_stage2= data.sort_values(by='Máy trải')
display(data_stage2)
#Gán việc cho máy cắt
cutting_machine = []
for ind in data_stage2.index:
    cutting_machine.append((data_stage2['Máy trải'][ind] + 1) // 2)

data_stage2['Máy cắt'] = cutting_machine
data_stage2 = data_stage2.sort_values(by=['Finish time 1', 'Máy cắt'])
display(data_stage2)
#Tính thời gian cho trạm cắt
num_cutting_machine = 4
cur_cutting_time: list[float] = [data_stage2[data_stage2['Máy cắt'] == idx + 1].iloc[0, 8] for idx in range(num_cutting_machine)]

data_stage2['Start time 2'] = 0.0
data_stage2['Finish time 2'] = 0.0

for ind in data_stage2.index:
    cutting_idx = data_stage2.at[ind, 'Máy cắt'] - 1
    data_stage2.at[ind, 'Start time 2'] = max(cur_cutting_time[cutting_idx], data_stage2.at[ind, 'Finish time 1'])
    data_stage2.at[ind, 'Finish time 2'] = data_stage2['Start time 2'][ind] + data_stage2['P cắt'][ind]

    # Update current cutting time
    cur_cutting_time[cutting_idx] = data_stage2.at[ind, 'Finish time 2']
#Tính độ trễ
data_stage2['Độ trễ (phút)']= np.maximum(0, data_stage2['Finish time 2'] - (data_stage2['Due date (h)'] * 60))
print('BẢNG KẾT QUẢ ĐIỀU ĐỘ TRẠM MÁY CẮT')
display(data_stage2)

#Tính hàm mục tiêu
column_count = 'Độ trễ (phút)'
num_tardiness = (data_stage2[column_count] > 0).sum()
print('Số công việc bị trễ theo luật FCFS = ', num_tardiness)
sum_tardiness = data_stage2['Độ trễ (phút)'].sum() 
print('Hàm mục tiêu theo luật FCFS = ',sum_tardiness,'phút')

# Thứ tự điều độ tại từng trạm
data_stage2
job_order_start_time_1 = data_stage2.sort_values(by='Start time 1')
print("\nKết quả thứ tự điều độ trên trạm trải:")
display(job_order_start_time_1[['Jobs', 'Start time 1']])

job_order_start_time_2 = data_stage2.sort_values(by='Start time 2')
print("\nKết quả thứ tự điều độ trên trạm cắt:")
display(job_order_start_time_2[['Jobs', 'Start time 2']])

,Jobs,Stockout Num,Total rolls,Total yards,Due date (h),P trải,P cắt,Start time 1,Finish time 1,Máy trải
0,15,20230720024151,5,144,9,19.20,24.0,0.00,86.14,1
17,17,20230720024153,7,225,9,30.00,33.6,86.14,130.41,1
9,7,20230720024143,8,587,2,78.27,38.4,47.20,140.27,2
1,13,20230720024149,6,196,9,26.14,28.8,0.00,47.20,2
13,14,20230720024150,5,275,2,36.67,24.0,64.80,101.47,3
2,12,20230720024148,6,315,4,42.00,28.8,0.00,64.80,3
3,18,20230720024154,9,332,4,44.27,43.2,0.00,44.94,4
12,6,20230720024142,5,354,7,47.20,24.0,60.01,86.15,4
8,9,20230720024145,5,113,2,15.07,24.0,44.94,60.01,4
18,16,20230720024152,8,251,2,33.47,38.4,86.15,182.15,4


,Jobs,Stockout Num,Total rolls,Total yards,Due date (h),P trải,P cắt,Start time 1,Finish time 1,Máy trải,Máy cắt
3,18,20230720024154,9,332,4,44.27,43.2,0.00,44.94,4,2
1,13,20230720024149,6,196,9,26.14,28.8,0.00,47.20,2,1
5,19,20230720024155,7,720,9,96.00,33.6,0.00,47.20,6,3
4,4,20230720024140,5,337,2,44.94,24.0,0.00,54.14,5,3
8,9,20230720024145,5,113,2,15.07,24.0,44.94,60.01,4,2
2,12,20230720024148,6,315,4,42.00,28.8,0.00,64.80,3,2
10,5,20230720024141,7,406,2,54.14,33.6,47.20,65.87,6,3
6,1,20230720024137,10,646,4,86.14,48.0,0.00,78.27,7,4
14,3,20230720024139,6,486,4,64.80,28.8,65.87,85.07,6,3
0,15,20230720024151,5,144,9,19.20,24.0,0.00,86.14,1,1


BẢNG KẾT QUẢ ĐIỀU ĐỘ TRẠM MÁY CẮT


,Jobs,Stockout Num,Total rolls,Total yards,Due date (h),P trải,P cắt,Start time 1,Finish time 1,Máy trải,Máy cắt,Start time 2,Finish time 2,Độ trễ (phút)
3,18,20230720024154,9,332,4,44.27,43.2,0.00,44.94,4,2,44.94,88.14,0.00
1,13,20230720024149,6,196,9,26.14,28.8,0.00,47.20,2,1,47.20,76.00,0.00
5,19,20230720024155,7,720,9,96.00,33.6,0.00,47.20,6,3,47.20,80.80,0.00
4,4,20230720024140,5,337,2,44.94,24.0,0.00,54.14,5,3,80.80,104.80,0.00
8,9,20230720024145,5,113,2,15.07,24.0,44.94,60.01,4,2,88.14,112.14,0.00
2,12,20230720024148,6,315,4,42.00,28.8,0.00,64.80,3,2,112.14,140.94,0.00
10,5,20230720024141,7,406,2,54.14,33.6,47.20,65.87,6,3,104.80,138.40,18.40
6,1,20230720024137,10,646,4,86.14,48.0,0.00,78.27,7,4,78.27,126.27,0.00
14,3,20230720024139,6,486,4,64.80,28.8,65.87,85.07,6,3,138.40,167.20,0.00
0,15,20230720024151,5,144,9,19.20,24.0,0.00,86.14,1,1,86.14,110.14,0.00


Số công việc bị trễ theo luật FCFS =  7
Hàm mục tiêu theo luật FCFS =  584.16 phút

Kết quả thứ tự điều độ trên trạm trải:


,Jobs,Start time 1
3,18,0.00
7,8,0.00
6,1,0.00
2,12,0.00
0,15,0.00
4,4,0.00
5,19,0.00
1,13,0.00
8,9,44.94
10,5,47.20



Kết quả thứ tự điều độ trên trạm cắt:


,Jobs,Start time 2
3,18,44.94
1,13,47.20
5,19,47.20
6,1,78.27
4,4,80.80
0,15,86.14
8,9,88.14
10,5,104.80
2,12,112.14
15,20,126.27


CODE GA

In [37]:
import pandas as pd
import numpy as np
import heapq
import random

# Hàm tạo NST từ luật EDD
def create_nst_EDD():
    data = pd.read_excel(os.path.join(data_path, 'Data.xlsx'), sheet_name=0, index_col=0)
    data = data.reset_index()
    data['Start time 1'] = 0
    data['Finish time 1'] = 0
    data['Máy trải'] = 0
    data['Máy cắt'] = 0
    data['Start time 2'] = 0.0
    data['Finish time 2'] = 0.0
    data['Độ trễ (phút)'] = 0

    # Sắp xếp công việc theo thời gian tới hạn (due date) tăng dần
    sorted_jobs = data.sort_values(by='Due date (h)').values

    # Khởi tạo hàng đợi ưu tiên (min heap) để theo dõi thời gian hoàn thành trên từng máy
    machines_heap = [(0, machine_id) for machine_id in range(8)]

    # Lập lịch công việc
    for job in sorted_jobs:
        processing_time, machine_id = heapq.heappop(machines_heap)
        start_time = max(processing_time, 0)
        finish_time = start_time + job[5]
        data.at[int(job[0]) - 1, 'Start time 1'] = start_time
        data.at[int(job[0]) - 1, 'Finish time 1'] = finish_time
        data.at[int(job[0]) - 1, 'Máy trải'] = machine_id + 1
        heapq.heappush(machines_heap, (finish_time, machine_id))

    # Gán việc cho máy cắt
    cutting_machine = []
    for ind in data.index:
        cutting_machine.append((data['Máy trải'][ind] + 1) // 2)
    data['Máy cắt'] = cutting_machine
    data = data.sort_values(by=['Finish time 1', 'Máy cắt'])

    # Tính thời gian cho trạm cắt
    num_cutting_machine = 4
    cur_cutting_time = [data[data['Máy cắt'] == idx + 1].iloc[0, 8] for idx in range(num_cutting_machine)]

    for ind in data.index:
        cutting_idx = data.at[ind, 'Máy cắt'] - 1
        data.at[ind, 'Start time 2'] = max(cur_cutting_time[cutting_idx], data.at[ind, 'Finish time 1'])
        data.at[ind, 'Finish time 2'] = data['Start time 2'][ind] + data['P cắt'][ind]
        data.at[ind, 'Độ trễ (phút)'] = max(0, data.at[ind, 'Finish time 2'] - (data.at[ind, 'Due date (h)'] * 60))
        cur_cutting_time[cutting_idx] = data.at[ind, 'Finish time 2']

    return data
# Hàm tạo NST từ luật FCFS
def create_nst_FCFS():
    data = pd.read_excel(os.path.join(data_path, 'Data.xlsx'), sheet_name=0, index_col=0)
    data = data.reset_index()
    data['Start time 1'] = 0
    data['Finish time 1'] = 0
    data['Máy trải'] = 0
    data['Máy cắt'] = 0
    data['Start time 2'] = 0.0
    data['Finish time 2'] = 0.0
    data['Độ trễ (phút)'] = 0

    # Sắp xếp công việc theo FCFS
    sorted_jobs = data.sort_values(by='Jobs').values

    # Khởi tạo hàng đợi ưu tiên (min heap) để theo dõi thời gian hoàn thành trên từng máy
    machines_heap = [(0, machine_id) for machine_id in range(8)]

    # Lập lịch công việc
    for job in sorted_jobs:
        processing_time, machine_id = heapq.heappop(machines_heap)
        start_time = max(processing_time, 0)
        finish_time = start_time + job[5]
        data.at[int(job[0]) - 1, 'Start time 1'] = start_time
        data.at[int(job[0]) - 1, 'Finish time 1'] = finish_time
        data.at[int(job[0]) - 1, 'Máy trải'] = machine_id + 1
        heapq.heappush(machines_heap, (finish_time, machine_id))

    # Gán việc cho máy cắt
    cutting_machine = []
    for ind in data.index:
        cutting_machine.append((data['Máy trải'][ind] + 1) // 2)
    data['Máy cắt'] = cutting_machine
    data = data.sort_values(by=['Finish time 1', 'Máy cắt'])

    # Tính thời gian cho trạm cắt
    num_cutting_machine = 4
    cur_cutting_time = [data[data['Máy cắt'] == idx + 1].iloc[0, 8] for idx in range(num_cutting_machine)]

    for ind in data.index:
        cutting_idx = data.at[ind, 'Máy cắt'] - 1
        data.at[ind, 'Start time 2'] = max(cur_cutting_time[cutting_idx], data.at[ind, 'Finish time 1'])
        data.at[ind, 'Finish time 2'] = data['Start time 2'][ind] + data['P cắt'][ind]
        data.at[ind, 'Độ trễ (phút)'] = max(0, data.at[ind, 'Finish time 2'] - (data.at[ind, 'Due date (h)'] * 60))
        cur_cutting_time[cutting_idx] = data.at[ind, 'Finish time 2']

    return data
# Hàm tạo NST từ luật SPT
def create_nst_SPT():
    data = pd.read_excel('./Data.xlsx', sheet_name=0, index_col=0)
    data = data.reset_index()
    data['Start time 1'] = 0
    data['Finish time 1'] = 0
    data['Máy trải'] = 0
    data['Máy cắt'] = 0
    data['Start time 2'] = 0.0
    data['Finish time 2'] = 0.0
    data['Độ trễ (phút)'] = 0

    # Sắp xếp công việc theo thời gian P trải tăng dần
    sorted_jobs = data.sort_values(by='P trải').values

    # Khởi tạo hàng đợi ưu tiên (min heap) để theo dõi thời gian hoàn thành trên từng máy
    machines_heap = [(0, machine_id) for machine_id in range(8)]

    # Lập lịch công việc
    for job in sorted_jobs:
        processing_time, machine_id = heapq.heappop(machines_heap)
        start_time = max(processing_time, 0)
        finish_time = start_time + job[5]
        data.at[int(job[0]) - 1, 'Start time 1'] = start_time
        data.at[int(job[0]) - 1, 'Finish time 1'] = finish_time
        data.at[int(job[0]) - 1, 'Máy trải'] = machine_id + 1
        heapq.heappush(machines_heap, (finish_time, machine_id))

    # Gán việc cho máy cắt
    cutting_machine = []
    for ind in data.index:
        cutting_machine.append((data['Máy trải'][ind] + 1) // 2)
    data['Máy cắt'] = cutting_machine
    data = data.sort_values(by=['Finish time 1', 'Máy cắt'])

    # Tính thời gian cho trạm cắt
    num_cutting_machine = 4
    cur_cutting_time = [data[data['Máy cắt'] == idx + 1].iloc[0, 8] for idx in range(num_cutting_machine)]

    for ind in data.index:
        cutting_idx = data.at[ind, 'Máy cắt'] - 1
        data.at[ind, 'Start time 2'] = max(cur_cutting_time[cutting_idx], data.at[ind, 'Finish time 1'])
        data.at[ind, 'Finish time 2'] = data['Start time 2'][ind] + data['P cắt'][ind]
        data.at[ind, 'Độ trễ (phút)'] = max(0, data.at[ind, 'Finish time 2'] - (data.at[ind, 'Due date (h)'] * 60))
        cur_cutting_time[cutting_idx] = data.at[ind, 'Finish time 2']

    return data
# Hàm tạo NST từ luật LPT
def create_nst_LPT():
    data = pd.read_excel('./Data.xlsx', sheet_name=0, index_col=0)
    data = data.reset_index()
    data['Start time 1'] = 0
    data['Finish time 1'] = 0
    data['Máy trải'] = 0
    data['Máy cắt'] = 0
    data['Start time 2'] = 0.0
    data['Finish time 2'] = 0.0
    data['Độ trễ (phút)'] = 0

    # Sắp xếp công việc theo thời gian P trải giảm dần
    sorted_jobs = data.sort_values(by='P trải', ascending=False).values

    # Khởi tạo hàng đợi ưu tiên (min heap) để theo dõi thời gian hoàn thành trên từng máy
    machines_heap = [(0, machine_id) for machine_id in range(8)]

    # Lập lịch công việc
    for job in sorted_jobs:
        processing_time, machine_id = heapq.heappop(machines_heap)
        start_time = max(processing_time, 0)
        finish_time = start_time + job[5]
        data.at[int(job[0]) - 1, 'Start time 1'] = start_time
        data.at[int(job[0]) - 1, 'Finish time 1'] = finish_time
        data.at[int(job[0]) - 1, 'Máy trải'] = machine_id + 1
        heapq.heappush(machines_heap, (finish_time, machine_id))

    # Gán việc cho máy cắt
    cutting_machine = []
    for ind in data.index:
        cutting_machine.append((data['Máy trải'][ind] + 1) // 2)
    data['Máy cắt'] = cutting_machine
    data = data.sort_values(by=['Finish time 1', 'Máy cắt'])

    # Tính thời gian cho trạm cắt
    num_cutting_machine = 4
    cur_cutting_time = [data[data['Máy cắt'] == idx + 1].iloc[0, 8] for idx in range(num_cutting_machine)]

    for ind in data.index:
        cutting_idx = data.at[ind, 'Máy cắt'] - 1
        data.at[ind, 'Start time 2'] = max(cur_cutting_time[cutting_idx], data.at[ind, 'Finish time 1'])
        data.at[ind, 'Finish time 2'] = data['Start time 2'][ind] + data['P cắt'][ind]
        data.at[ind, 'Độ trễ (phút)'] = max(0, data.at[ind, 'Finish time 2'] - (data.at[ind, 'Due date (h)'] * 60))
        cur_cutting_time[cutting_idx] = data.at[ind, 'Finish time 2']

    return data

# Hàm tạo NST ngẫu nhiên
def create_random_nst():
    data = pd.read_excel('./Data.xlsx', sheet_name=0, index_col=0)
    data = data.reset_index()
    data['Start time 1'] = 0
    data['Finish time 1'] = 0
    data['Máy trải'] = 0
    data['Máy cắt'] = 0
    data['Start time 2'] = 0.0
    data['Finish time 2'] = 0.0
    data['Độ trễ (phút)'] = 0

    # Sắp xếp công việc ngẫu nhiên
    randomly_sorted_jobs = data.sample(frac=1).values

    machines_heap = [(0, machine_id) for machine_id in range(8)]

    for job in randomly_sorted_jobs:
        processing_time, machine_id = heapq.heappop(machines_heap)
        start_time = max(processing_time, 0)
        finish_time = start_time + job[5]
        data.at[int(job[0]) - 1, 'Start time 1'] = start_time
        data.at[int(job[0]) - 1, 'Finish time 1'] = finish_time
        data.at[int(job[0]) - 1, 'Máy trải'] = machine_id + 1
        heapq.heappush(machines_heap, (finish_time, machine_id))

    cutting_machine = []
    for ind in data.index:
        cutting_machine.append((data['Máy trải'][ind] + 1) // 2)
    data['Máy cắt'] = cutting_machine
    data = data.sort_values(by=['Finish time 1', 'Máy cắt'])

    num_cutting_machine = 4
    cur_cutting_time = [data[data['Máy cắt'] == idx + 1].iloc[0, 8] for idx in range(num_cutting_machine)]

    for ind in data.index:
        cutting_idx = data.at[ind, 'Máy cắt'] - 1
        data.at[ind, 'Start time 2'] = max(cur_cutting_time[cutting_idx], data.at[ind, 'Finish time 1'])
        data.at[ind, 'Finish time 2'] = data['Start time 2'][ind] + data['P cắt'][ind]
        data.at[ind, 'Độ trễ (phút)'] = max(0, data.at[ind, 'Finish time 2'] - (data.at[ind, 'Due date (h)'] * 60))
        cur_cutting_time[cutting_idx] = data.at[ind, 'Finish time 2']

    return data

# Hàm tạo quần thể ban đầu
def create_initial_population(population_size):
    # Khởi tạo quần thể
    population = []
    population.extend([create_nst_EDD(), create_nst_EDD(), create_nst_EDD(), create_nst_EDD()])
    # Thêm 6 NST tạo ngẫu nhiên
    for _ in range(6):
        population.append(create_random_nst())
    return population
# Hàm tính Hàm mục tiêu của mỗi NST
def objective(nst):
    data_stage2 = nst.copy()
    column_count = 'Độ trễ (phút)'
    num_tardiness = (data_stage2[column_count] > 0).sum()
    sum_tardiness = data_stage2['Độ trễ (phút)'].sum()
    return sum_tardiness
# Hàm tính độ thích nghi của mỗi NST
def fitness(nst, max_objective):
    sum_tardiness = nst['Độ trễ (phút)'].sum()
    return max_objective - sum_tardiness
# Hàm chọn lọc theo phương pháp Roulette Wheel
def roulette_wheel_selection(population, num_parents):
    max_objective = max([objective(nst) for nst in population])
    fitness_values = [fitness(nst, max_objective) for nst in population]
    total_fitness = sum(fitness_values)
    probabilities = [fit / total_fitness for fit in fitness_values]
    probabilities_prefix:list[float] = []
    for i in range(len(probabilities)):
        probabilities_prefix.append(probabilities[i] + (probabilities_prefix[-1] if i != 0 else 0))
    
    random_list = [random.random() for _ in range(len(population))]
    chosen_parents = [population[i] for i in np.searchsorted(probabilities_prefix, random_list, side='right')]
    print(chosen_parents)
    return chosen_parents

# Hàm lai chéo Precedence Operation Crossover (POX)
def precedence_operation_crossover(parent1, parent2):
    # (Xác định cắt ở mức độ gen hoặc cụm gen, tùy thuộc vào bài toán cụ thể)
    crossover_point = random.randint(0, len(parent1) - 20)

    child1 = parent1.copy()
    child2 = parent2.copy()

    # Áp dụng POX
    child1[:crossover_point] = parent2[:crossover_point]
    child2[:crossover_point] = parent1[:crossover_point]

    return child1, child2

# Hàm đột biến Reciprocal Exchange Mutation (REM)
def reciprocal_exchange_mutation(nst):
    mutation_point_1 = random.randint(1, len(nst) // 2)
    mutation_point_2 = random.randint(0, mutation_point_1)

    mutated_nst = nst.copy()

    # Áp dụng REM
    mutated_nst[mutation_point_1], mutated_nst[mutation_point_2] = mutated_nst[mutation_point_2].copy(), mutated_nst[mutation_point_1].copy()

    return mutated_nst

# Hàm thay thế theo Acceptance Threshold
def acceptance_threshold_replacement(population, offspring):
    # Số lượng NST trong quần thể
    P = len(population)
    #Ngưỡng chấp nhận là NST thứ K
    k = P // 2

    # Tính hàm mục tiêu cho từng NST con
    offspring_objective = [objective(nst) for nst in offspring]# list[int]
    
    # Sắp xếp NST con theo hàm mục tiêu giảm dần
    sorted_offspring_indices = np.argsort(offspring_objective)[::-1]

    # Chọn NST thứ k trong quần thể hiện tại để so sánh
    kth_population_objective = objective(population[k - 1])

    # Thêm NST con vào quần thể nếu hàm mục tiêu của NST con tốt hơn NST thứ k
    for i, nst_index in enumerate(sorted_offspring_indices):
        if objective(offspring[nst_index]) < kth_population_objective:
            population[k - 1] = offspring[nst_index]
            break
    #Loại bỏ NST có hàm mục tiêu xấu nhất
    nst_remove = np.argmax([objective(nst) for nst in population])
    population.pop(nst_remove )
    return population


# Hàm chạy thuật toán di truyền
def genetic_algorithm(num_generations=50, population_size=10, crossover_probability=0.8, mutation_probability=0.2):
    # Tạo quần thể ban đầu
    population = create_initial_population(population_size)

    for generation in range(num_generations):
        # Chọn lọc
        parents = roulette_wheel_selection(population, num_parents=population_size)



        # Lai chéo
        offspring = []
        for parent1, parent2 in zip(parents[::2], parents[1::2]):
            parent1, parent2 = parent1.tolist(), parent2.tolist()
            print(parent1)
            if random.random() < crossover_probability:
                child1, child2 = precedence_operation_crossover(parent1, parent2)
                offspring.extend([child1, child2])
            else:
                offspring.extend([parent1, parent2])
        
        
        # Đột biến
        for i in range(len(offspring)):
            if random.random() < mutation_probability:
                offspring[i] = reciprocal_exchange_mutation(offspring[i])
        print('abc')
        # Thay thế
        population = acceptance_threshold_replacement(population, offspring)

        # In thông tin về hàm mục tiêu của NST tốt nhất trong thế hệ hiện tại
        best_nst = min(population, key=objective)
        best_objective = objective(best_nst)
        print(f"Generation {generation + 1}, Best Objective: {best_objective}")

    # Trả về NST tốt nhất sau khi chạy xong thuật toán
    return min(population, key=objective)


# Gọi hàm chạy thuật toán di truyền
genetic_algorithm()


[    Jobs    Stockout Num  Total rolls  Total yards  Due date (h)  P trải   
8      9  20230720024145            5          113             2   15.07  \
15    16  20230720024152            8          251             2   33.47   
13    14  20230720024150            5          275             2   36.67   
3      4  20230720024140            5          337             2   44.94   
4      5  20230720024141            7          406             2   54.14   
1      2  20230720024138            7          354             2   47.20   
19    20  20230720024156            9          543             2   72.40   
11    12  20230720024148            6          315             4   42.00   
6      7  20230720024143            8          587             2   78.27   
17    18  20230720024154            9          332             4   44.27   
10    11  20230720024147            5          140             7   18.67   
9     10  20230720024146           10          698             2   93.07   
2      3  2

AttributeError: 'DataFrame' object has no attribute 'tolist'